### `파이썬 버전 관련`
- 인자의 타입 힌트를 list[str] 등으로 주는 방식 -> `3.9버전 이상`부터 가능
    - 예) def _expand_composition_data(self, comz_tokens: list[str]) -> list[int]:

`수정 이력`
- 8월 14일 버전: MasterParser() 클래스 기능 구현 완료
- 8월 18일 버전: DeckModifier() 클래스 구현 시작
- 8월 27일 버전: %GEN_DIM에 값이 반영되도록 수정
    - Parser에서 'GEN_DIM' 전용 메서드 작성
    - Modifier에서 nbatch 및 ncomp를 읽은 후 GEN_DIM을 마지막으로 수정하도록 해야함
    - Writer에서 'GEN_DIM'을 반영하도록 수정

In [ ]:
import os
import copy # modifier 클래스에서 사용
import sys # writer 클래스에서 사용
from itertools import groupby # writer 클래스에서 사용
import random # modifier 클래스에서 사용

current_dir = "C:/Users/fnc/Desktop/src/github.com/JSK3226/aicorekit-py/datainformed" # 지금은 git 리포지토리 내부에 작성하지만, 완전히 다 작성한 후에는 파일만 따로 이동시켜 매크로 동작할 예정
mas_io_dir = "C:/Users/user/Desktop/src/github.com/JSK3226/aicorekit-py/datainformed/MASTER_IO"

os.chdir(mas_io_dir)

print("- 현재 작업 폴더: \n \t", os.getcwd(),"\n")

print("- 파일/폴더 목록:")
# 파일/폴더 목록
for name in os.listdir():
    print("\t",name)

- 현재 작업 폴더: 
 	 C:\Users\user\Desktop\src\github.com\JSK3226\aicorekit-py\datainformed\MASTER_IO 

- 파일/폴더 목록:
	 DEP_iSMR_DPL1_F01 - 복사본.INP
	 DEP_iSMR_DPL1_F01.INP
	 DEP_iSMR_DPL1_F02.INP
	 DEP_iSMR_DPL1_F03.INP
	 DEP_iSMR_DPL1_F04.INP
	 DEP_iSMR_DPL1_F05.INP
	 DEP_iSMR_DPL1_F06.INP
	 DEP_iSMR_DPL1_F07.INP
	 DEP_iSMR_DPL1_F08.INP
	 iSMR_DPL1_HFF
	 iSMR_DPL1_XSL_KNFREFL
	 MASTER4.0_UM_rev04.pdf
	 MAS_HFF
	 MAS_INP
	 MAS_INP0
	 MAS_INPe
	 MAS_INP_loop0
	 MAS_INP_loop1
	 MAS_OUT
	 MAS_OUT_loop0
	 MAS_OUT_loop1
	 MAS_RST.DEPF_01_0000.00_C
	 MAS_RST.DEPF_01_0005.00_C
	 MAS_RST.DEPF_01_0015.00_C
	 MAS_RST.DEPF_01_0030.00_C
	 MAS_RST.DEPF_01_0060.00_C
	 MAS_RST.DEPF_01_0090.00_C
	 MAS_RST.DEPF_01_0120.00_C
	 MAS_SUM
	 MAS_SUM_loop0
	 MAS_XSL
	 run_mas.exe


클래스
- Deck
- DeckParser
  - MasterParser
  - AstraParser
- DeckModifier
  - MasterModifier
  - AstraModifier
- DeckWriter
  - MasterWriter
  - AstraWriter
- SimulationRunner
  - MasterRunner
  - AstraRunner
- DataGenerationPipeline

## 1. Deck

In [1]:
class Deck:
    def __init__(self):
        # 파일에 나타나는 카드 '타입'의 순서를 그대로 저장
        # 예: ['GEN_SYM', 'EXE_DEP', 'EXE_OPT', 'EXE_DEP']
        self.card_sequence = []
        
        # 카드 '타입'을 키로, 파싱된 내용의 '리스트'를 값으로 저장
        # 예: {'EXE_DEP': [내용A, 내용B], 'GEN_SYM': [내용C]}
        self.card_data = {}

## 2. DeckParser
- MasterParser(DeckParser)
- AstraParser(DeckParser)

In [4]:
class DeckParser:
    def parse_input_file(self, base_file_path):
        raise NotImplementedError

1a-1: MasterParser
- 베이스 입력덱 파일을 입력받아 Deck() 인스턴스를 채운 후 반환

In [5]:
class MasterParser(DeckParser):
    def __init__(self):
        # 특정 카드('%')에 대한 파싱함수 목록
            # 해당 카드들은 DeckEditor 및 Writer에서 전용 파싱함수로 처리되어야 함
        self.parsing_method = {
            'GEN_SYM': self._parse_gen_sym,
            'GEN_DIM': self._parse_gen_dim,
            'LPD_BCH': self._parse_lpd_bch,
            'LPD_B&C': self._parse_lpd_b_and_c, 
            'LPD_C&X': self._parse_lpd_c_and_x,
            'LPD_HFF': self._parse_lpd_hff,
            'EXE_DEP': self._parse_exe_dep,
        }

    def parse_input_file(self, base_file_path: str) -> Deck:
        deck = Deck()
        try:
            with open(base_file_path, 'r', encoding='utf-8') as f:
                line_iterator = iter(f)
                line = next(line_iterator, None) # next
            
                while line is not None: # 공백 라인은 '\n'이므로 None이 아님
                    
                    line_content = line.split('#', 1)[0].strip() # 주석(#) 및 공백 처리      
                           
                    if not line_content:
                        line = next(line_iterator, None)
                        continue
                    
                    elif line_content.startswith('%'):
                        card_name = line_content.lstrip('%').strip()
                        deck.card_sequence.append(card_name)
                        deck.card_data.setdefault(card_name, [])
                        
                        parse_function = self.parsing_method.get(card_name, self._parse_generic_cards)
                        parsed_content, next_line_to_process = parse_function(line_iterator)
                    
                        deck.card_data[card_name].append(parsed_content)
                        
                        # 하위 파서가 다음 라인을 반환했으면 그것을 사용하고,
                        # 그렇지 않으면(None이면) 메인 루프가 직접 다음 라인을 가져옴
                        if next_line_to_process is not None:
                            line = next_line_to_process
                        else:
                            line = next(line_iterator, None)
                    
                    elif line_content == '/':
                        card_name = '/'
                        deck.card_sequence.append(card_name)
                        deck.card_data.setdefault(card_name, []).append({'raw_lines': [line_content]})
                        line = next(line_iterator, None) # 다음 라인으로 명시적으로 이동
                        
                    else:
                        line = next(line_iterator, None)
                        
        except FileNotFoundError:
            print(f"[오류] 입력 파일을 찾을 수 없습니다: '{base_file_path}'")
            return None # 오류 발생 시 None 반환
        except Exception as e:
            print(f"[오류] 파일 파싱 중 알 수 없는 문제가 발생했습니다: {e}")
            return None
                
        return deck

    # -- 이하 파싱함수 목록
        # A. _parse_generic_cards
        # B. _parse_gen_sym
        # C. _parse_gen_dim
        # D. _parse_lpd_bch
        # E. _parse_lpd_b_and_c 
        # F. _parse_lpd_c_and_x
        # G. _parse_lpd_hff  
        # H. _parse_exe_dep

    ## A. _parse_generic_cards ##
    def _parse_generic_cards(self, line_iterator):
        """
        이터레이터를 받은 후, 다음 '%' 기호가 나타날 때 까지 원본(raw)라인을 추출한 뒤
        내용 및 이터레이터를 반환하는 메서드
        
        Args:
            line_iterator: 카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).
        Returns:
            dict[str, list[str]]: 수집한 라인 전체의 원본을 밸류값으로 가지는 딕셔너리
            line : '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
        
        """
        card_contents = []
        
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            if line_content.startswith(('%','/')):
                return {'raw_lines': card_contents}, line
            
            if line_content == 'END':
                return {'raw_lines': card_contents} , None

            # 해당 구문을 통해 공백라인 해결가능
            if line_content:
                card_contents.append(line) # 원본 내용을 추가하기 위해 line을 리스트에 추가
                
        # for 루프가 정상적으로 끝난경우, 파일의 마지막에 도달함을 의미
        # 지금까지 수집한 내용 및 더이상 라인이 없음(None)을 반환        
        return {'raw_lines': card_contents}, None
                
                
                
    ## B. _parse_gen_sym (%GEN_SYM) ##
    def _parse_gen_sym(self, line_iterator):
        """
        %GEN_SYM 입력카드에 포함된 두개의 줄, 6개의 숫자를 읽어 각각의 변수에 저장
        읽은 데이터는 self의 속성값(isymlx 등)에 저장하고 파싱 결과를 content로 반환
            
        Args:
            line_iterator (iter): 카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).

        Returns:
            tuple[dict, str | None]:
                - (파싱 결과 딕셔너리, 다음 처리를 위한 라인)
                
        Examples:
            - 입력덱 형식:
                %GEN_SYM
                isymlx  isymly  isymlz # x, y, z 방향의 왼쪽(Left) 경계 조건
                isymrx  isymry  isymrz # x, y, z 방향의 오른쪽(Right) 경계 조건

            - 설명:
                isymlx 및 isymly가 모두 -1일 때는 quater core이고 모두 0일때는 full core (2-D or 3-D)
                isymlz 및 isymrz가 모두  1일 때는 2-D, 모두 0일 때는 3-D 
        """
        
        # 입력카드에 존재하는건 6개의 정수형 값 뿐임
        # geometry_type 및 core_dimension은 6개의 값에 대해 조건문을 걸어 저장

        integer_set_list = []
        next_line_for_main_loop = None
        
        # 1. 해당 카드 내부를 순회하여 숫자로 이루어진 라인을 수집 후 리스트에 저장(append)
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            if line_content.upper() == 'END':
                print("[안내] 'END' 문자를 확인하여 %GEN_SYM 카드 처리를 종료합니다.")
                next_line_for_main_loop = None
                break
            
            if not line_content:
                continue
            
            if len(integer_set_list) <2: # 빈 리스트의 길이는 '0'임, 2번째 줄 까지 채워 넣어야 함
                is_valid_integer_line = False
                
                try:
                    integer_values_in_line = [int(num) for num in line_content.split()]
                    
                    # 정수가 3개일 경우에만 추가
                    if len(integer_values_in_line) == 3:
                        integer_set_list.append(integer_values_in_line)
                        is_valid_integer_line = True
                
                except ValueError:
                    # 정수가 아닌 숫자로 인해 num의 변환이 실패할 경우 is_valid_integer_line은 False로 유지
                    pass
                
                if not is_valid_integer_line: # True가 아닐 경우      
                    return {
                         'error': f"[오류] %GEN_SYM 카드에서 잘못된 데이터 형식이 발견되었습니다. 3개의 정수가 필요합니다. (라인: '{line.strip()}')"
                    }, None # 다음 라인 정보는 의미 없으므로 None              
                
        # 2. 루프 종료 후 수집된 데이터 처리
        if len(integer_set_list) != 2: # 성공적으로 완료되었다면 integer_set_list의 길이는 2임, 건너뜀
            return {
                'error': f"[오류] %GEN_SYM 카드에서 2줄의 정수 데이터가 필요하지만 {len(integer_set_list)}줄만 발견되었습니다."
            }, next_line_for_main_loop
            
        isymlx, isymly, isymlz = integer_set_list[0]
        isymrx, isymry, isymrz = integer_set_list[1]
        
        # 2a) 노심의 geometry_type 결정
        if isymlx == -1 and isymly == -1:
            geometry_type = 'QUARTER_CORE'
        elif isymlx == 0 and isymly == 0:
            geometry_type = 'FULL_CORE'
        else:
            geometry_type = 'UNKNOWN'
            
        # 2b) core_dimension 결정
        if isymlz == 1 and isymrz == 1:
            core_dim = '2D'
        elif isymlz == 0 and isymrz == 0:
            core_dim = '3D'
        else:
            core_dim = 'UNKNOWN'
            
        parsed_content = {
            'left_boundary_condition': [isymlx, isymly, isymlz],
            'right_boundary_condition': [isymrx, isymry, isymrz],
            'geometry_type': geometry_type,
            'core_dimension': core_dim
        }
        
        return parsed_content, next_line_for_main_loop #  next_line_for_main_loop의 경우 다음 '%' 카드 이름이 할당된 상태
            
            
    ## C. _parse_gen_dim (%GEN_DIM) ##
    def _parse_gen_dim(self, line_iterator):
        """
        '%GEN_DIM' 입력카드의 데이터를 포맷에 맞게 추출하여 반환하는 메서드.
        해당 카드에서 정의된 값들은 %LPD_BCH 및 %LPD_C&X 등과 일치하여야 함 (Modifier에서 검증)

        Args:
            line_iterator (iter): 카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).
            
        Returns:
            parsed_content (dict)
            next_line_for_main_loop (str): '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
        
        Examples:
            %GEN_DIM
                11      11      26      10      12   # nx,ny,nz,nbatch,ncomp,
                3       1       0       2       1    # ndim,ngeo,nsym,ndivxy,ndivz,
                2                             
        """
        
        data_lines = []
        next_line_for_main_loop = None
        
        # 1. 다음 '%' 카드나 'END'가 나오기 전까지, 주석과 공백을 제외한 모든 데이터 라인을 수집
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            if line_content.upper() == 'END':
                next_line_for_main_loop = None
                break
            
            # 내용이 있는 라인만 리스트에 추가합니다.
            if line_content:
                data_lines.append(line_content)

        # 2. 수집된 데이터 라인이 정확히 3줄인지 확인합니다.
        if len(data_lines) != 3:
            raise ValueError(f"[오류] %GEN_DIM 카드 형식이 올바르지 않습니다. 3줄의 데이터가 필요하지만 {len(data_lines)}줄을 찾았습니다.")

        # 3. 각 라인을 순서대로 파싱하고 변수에 할당합니다.
        try:
            # 첫 번째 줄 파싱 (5개 정수)
            line1_vals = [int(v) for v in data_lines[0].split()]
            if len(line1_vals) != 5:
                raise ValueError(f"[오류] %GEN_DIM 카드의 첫 번째 줄에는 5개의 정수가 필요하지만 {len(line1_vals)}개가 있습니다.")
            nx, ny, nz, nbatch, ncomp = line1_vals

            # 두 번째 줄 파싱 (5개 정수)
            line2_vals = [int(v) for v in data_lines[1].split()]
            if len(line2_vals) != 5:
                raise ValueError(f"[오류] %GEN_DIM 카드의 두 번째 줄에는 5개의 정수가 필요하지만 {len(line2_vals)}개가 있습니다.")
            ndim, ngeo, nsym, ndivxy, ndivz = line2_vals

            # 세 번째 줄 파싱 (1개 정수)
            line3_vals = [int(v) for v in data_lines[2].split()]
            if len(line3_vals) != 1:
                raise ValueError(f"[오류] %GEN_DIM 카드의 세 번째 줄에는 1개의 정수가 필요하지만 {len(line3_vals)}개가 있습니다.")
            ng = line3_vals[0]

        except ValueError as e:
            # int() 변환 실패 또는 개수 불일치 오류를 모두 처리합니다.
            raise ValueError(f"[오류] %GEN_DIM 카드 파싱 중 오류 발생: {e}")

        # 4. 파싱된 데이터를 딕셔너리로 만듭니다.
        parsed_content = {
            'nx': nx, 'ny': ny, 'nz': nz,
            'nbatch': nbatch, 'ncomp': ncomp,
            'ndim': ndim, 'ngeo': ngeo, 'nsym': nsym,
            'ndivxy': ndivxy, 'ndivz': ndivz, 'ng': ng
        }
        
        return parsed_content, next_line_for_main_loop
            
            
    ## D. _parse_lpd_bch (%LPD_BCH) ##
    def _parse_lpd_bch(self, line_iterator):
        """
        MASTER 입력 파일의 '%LPD_BCH' 카드에서 장전모형(loading pattern)을 추출하는 메서드
        '%LPD_BCH' 카드 블록 내의 모든 라인을 읽어 2차원 리스트(행렬)를 생성

        Args:
            line_iterator (iter): 카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).
        
        Returns:
            parsed_content dict[str,list[str]]: n x n 2차원 장전모형 행렬
            next_line_for_main_loop (str): '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
        
        Examples:
            %LPD_BCH
            #   (ibat2d(i,j),i=1,nx),j=1,ny
                o   o   R4  R2  R2  R1  R2  R2  R4  o   o 
                o   R6  R3  A1  A2  A2  A2  A1  R3  R6  o 
                R4  R3  A1  A2  A4  A2  A4  A2  A1  R3  R4
                R2  A1  A2  A4  A4  A4  A4  A4  A2  A1  R2
                R2  A2  A4  A4  A5  A5  A5  A4  A4  A2  R2
                R1  A2  A2  A4  A5  A4  A5  A4  A2  A2  R1
                R2  A2  A4  A4  A5  A5  A5  A4  A4  A2  R2
                R2  A1  A2  A4  A4  A4  A4  A4  A2  A1  R2
                R4  R3  A1  A2  A4  A2  A4  A2  A1  R3  R4
                o   R6  R3  A1  A2  A2  A2  A1  R3  R6  o 
                o   o   R4  R2  R2  R1  R2  R2  R4  o   o     
            
        """
        next_line_for_main_loop = None
        lp_matrix = []
        
        # 1. 루프를 통한 loading pattenr의 행(rows) 수집
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            if line_content.upper() == 'END':
                print("[안내] 'END' 문자를 확인하여 %LPD_BCH 카드 처리를 종료합니다.")
                next_line_for_main_loop = None
                break
            
            if not line_content:
                continue
            
            row_in_lp = line_content.split()
            lp_matrix.append(row_in_lp)
            
        # 2. 루프 종료 후 수집된 데이터 검증
        if not lp_matrix:
            raise ValueError("[오류] %LPD_BCH 카드에서 장전모형 데이터가 발견되지 않았습니다.")
        
        # 첫 행의 길이
        len_first_row = len(lp_matrix[0])
        
        # 2a) 모든행의 열이 동일한지 확인
        if not all(len(row) == len_first_row for row in lp_matrix):
            raise ValueError("[오류] %LPD_BCH 카드의 각 행이 동일한 열 개수를 가져야 합니다.")
            
        # 2b) 정방형인지 검증
        if len(lp_matrix) != len_first_row:
            raise ValueError(
                f"[오류] %LPD_BCH 카드가 정방행렬(n x n)이 아닙니다. 행: {len(lp_matrix)}, 열: {len_first_row}"
            )
            
        parsed_content = {'loading_pattern': lp_matrix}
        
        return parsed_content, next_line_for_main_loop
        
        
        
    ## E. _parse_lpd_b_and_c (%LPD_B&C) ##
    def _parse_lpd_b_and_c(self, line_iterator):
        """%LPD_B&C 카드를 파싱하여 각 연료 배치(batch)별 축 방향 조성 데이터를 추출, 딕셔너리화

        Args:
            line_iterator (iter): 카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).

        Returns:
            parsed_content (dict[str, list[int]]):
                            key = 배치 ID (예: "B1")
                            value = 확장된 축 방향 조성 값 리스트.
                
            next_line_for_main_loop (str): '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
            
        Examples:
            %LPD_B&C
                #       (ibatch(i),icomz(i,k),k=1,nz),i=1,nbatch
                R1         26*-2
                R2         26*-3
                R3         26*-4
                R4         26*-5
                R5         26*-6
                R6         26*-7
                A1      -1  21*1   3*5   -1
                A2      -1  21*2   3*5   -1
                A4      -1  21*3   3*5   -1
                A5      -1  21*4   3*5   -1   
        """
        next_line_for_main_loop = None
        parsed_content = {}
        
        for line in line_iterator:            
            # 주석 제거 후 내용만 취득(공백도 가능)
            line_content = line.split('#', 1)[0].strip()
            
            # 다음 카드('%') 만나면 루프 종료
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            # 'END'를 만나면 루프 종료
            if line_content.startswith('END'):
                print("\'END\' 문자를 확인하여 종료, (%LPD_B&C 카드)")
                next_line_for_main_loop = None
                break
                
            # 빈줄 건너뛰기 
            if not line_content:
                continue
            
            line_parts = line_content.split()
            batch_id = line_parts[0]
            axial_comp_tokens = line_parts[1:]
            
            expanded_axial_composition = self._expand_composition_tokens(axial_comp_tokens)
            
            if not expanded_axial_composition:
                print(f"[경고] %LPD_B&C 카드에서 배치 ID '{batch_id}'의 토큰 분할에 실패하여 건너뜁니다.")
                continue
            
            parsed_content[batch_id] = expanded_axial_composition # 딕셔너리에 저장
        
        return parsed_content, next_line_for_main_loop
    
            
    def _expand_composition_tokens(self, comp_tokens: list[str]) -> list[int]:
        """
        메서드 'parse_lpd_b_and_c'에서 사용
        N*V (count, value) 형태의 축약된 축 방향 조성 표현을 정수 리스트로 확장

        Args:
            comp_tokens (list[str]): 하나의 연료 ID(Batch ID)에 해당하는
                축 방향 조성(composition) 토큰들의 리스트.
                예시: ['26*-2'] 또는 ['-1', '21*1', '3*5', '-1']
           
        Returns:
            list[int]: 모든 축약 표현이 확장된, 최종적인 축 방향 조성 정수 리스트.
                       (리스트의 길이는 전체 축방향 노드(axial node)의 개수(nz)와 동일)
        """
        
        expanded_axial_composition = []
        
        # comp_tokens이 *을 포함하고 있다면 N*V 꼴이므로 N개만큼 V를 append
        # 아니면 그냥 append
        
        for token in comp_tokens:
            if '*' in token:
                try:
                    count, value = map(int, token.split('*'))
                    expanded_axial_composition.extend([value]*count)
                    
                except (ValueError, IndexError):
                    # 파싱 실패 시 오류 처리
                    print(f"[경고] %LPD_B&C 카드에서 토큰 '{token}'을(를) 해석할 수 없습니다.")
                    return [] # 빈 리스트 반환
            else:
                try:
                    expanded_axial_composition.append(int(token))
                except ValueError:
                    print(f"[경고] %LPD_B&C 카드에서 토큰 '{token}'을(를) 해석할 수 없습니다.")
                    
        return expanded_axial_composition
    
    
    
    ## F. _parse_lpd_c_and_x (%LPD_C&X) ##
    def _parse_lpd_c_and_x(self, line_iterator):
        """
        %LPD_C&X 카드의 내용을 파싱하여 composition index와 composition name의 매핑 딕셔너리를 반환하는 메서드.

        Args:
            line_iterator (iter):  카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).

        Returns:
            parsed_content (dict[int, str]):
                key = composition index (예: -1, 0, 1, 2, ...)
                value = composition name (예: "COMPOSITION_1")
                
            next_line_for_main_loop (str): '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
            
        Examples:
            %LPD_C&X
            #       icomp(i),nmxset(i),ind(i),nfrods(i),i=1,ncomp
            #       (idnucl(i,j),xnd(i,j),j=1,ind(i)),i=1,ncomp
                    -1      REFLECTOR_AX       
                    -2      REFLECTOR_R1                   
                    -3      REFLECTOR_R2                    
                    -4      REFLECTOR_R3          
                    -5      REFLECTOR_R4          
                    -6      REFLECTOR_R5          
                    -7      REFLECTOR_R6          
                    1       400_500_000               
                    2       400_501_001              
                    3       400_700_001             
                    4       400_700_501            
                    5       200_100_020             
        """
        
        # 읽은 라인을 split 한 이후에 처음 정수(integer)를 key로, 이후 composition_name을 value로
        next_line_for_main_loop = None
        parsed_content = {}
        
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            # 다음 카드('%') 만나면 루프 종료 (근데 왜 바로 return 안하고 루프를 종료?)
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            # 'END'를 만나면 루프 종료
            if line_content.startswith('END'):
                print("[안내] 'END' 문자를 확인하여 %LPD_C&X 카드 처리를 종료합니다.")
                next_line_for_main_loop = None
                break
            
            # 빈줄 건너뛰기
            if not line_content:
                continue
            
            line_parts = line_content.split()
            
            if len(line_parts) < 2:       
                raise ValueError(f"[오류] %LPD_C&X 카드에서 라인 형식이 잘못되었습니다. 최소 2개의 항목이 필요합니다. (발견된 항목 수: {len(line_parts)})")
     
            try:
                composition_index = int(line_parts[0])
                composition_name  = str(line_parts[1])
                
                parsed_content[composition_index] = composition_name
     
            except ValueError: 
                print(f"[오류] %LPD_C&X 카드에서 라인 '{line_content}'을(를) 해석할 수 없습니다.")
                
        return parsed_content, next_line_for_main_loop
                
                
    ## G. _parse_lpd_hff (%LPD_HFF) ##
    def _parse_lpd_hff(self, line_iterator):
        """
        %LPD_HFF 카드의 내용을 파싱하여 번호와 comp_name의 매핑 딕셔너리를 반환하는 메서드.

        Args:
            line_iterator (iter):  카드 내용의 첫 줄부터 시작하는 이터레이터(카드명(%) 미포함).

        Returns:
            parsed_content (dict[int, str]):
                key = 번호 (예: 1, 2, ...)
                value = comp_name (예: "400_516_000")
                
            next_line_for_main_loop (str): '%' 기호로 시작하는 다음 카드 라인(상위 루프에서 재사용).
            
        Examples:
            %LPD_HFF
            #       icomp(i),hff1(i),i=1,ncomp-nref
                    1       400_516_000
                    2       400_520_000
        """
        next_line_for_main_loop = None
        parsed_content = {}
        
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            
            if line_content.startswith('%'):
                next_line_for_main_loop = line
                break
            
            if line_content.startswith('END'):
                print("[안내] 'END' 문자를 확인하여 %LPD_HFF 카드 처리를 종료합니다.")
                next_line_for_main_loop = None
                break
            
            if not line_content:
                continue
            
            line_parts = line_content.split()
            
            if len(line_parts) < 2:
                print(f"[경고] %LPD_HFF 카드에서 형식이 잘못된 라인을 건너뜁니다: '{line_content}'")
                continue
     
            try:
                key_number = int(line_parts[0])
                comp_name  = str(line_parts[1])
                parsed_content[key_number] = comp_name
     
            except ValueError: 
                print(f"[경고] %LPD_HFF 카드에서 첫 번째 항목이 정수가 아닌 라인을 건너뜁니다: '{line_content}'")
                
        return parsed_content, next_line_for_main_loop
    

    ## H. _parse_exe_dep (%EXE_DEP) ##
    def _parse_exe_dep(self, line_iterator):
        next_line_for_main_loop = None
        parsed_content = {
            'depletion_interval': None, 'target_option': None,
            'search_parameter': None, 'search_value': None,
            'target_keff': None, 'target_boron': None,
        }
        
        for line in line_iterator:
            line_content = line.split('#', 1)[0].strip()
            if line_content.upper() == 'END':
                print("[안내] 'END' 문자를 확인하여 %EXE_DEP 카드 처리를 종료합니다.")
                next_line_for_main_loop = None
                break
            
            if line_content.startswith(('%', '/')):
                next_line_for_main_loop = line
                break
            
            if not line_content:
                continue
            
            try:
                line_parts = line_content.split()
                delt = float(line_parts[0])
                itg = int(line_parts[1])
                parsed_content['depletion_interval'] = delt
                parsed_content['target_option'] = itg
                
                if delt < 0:
                    next_line_raw = next(line_iterator, None)
                    if next_line_raw is None:
                        print(f"[경고] %EXE_DEP: delt < 0 이나, 이후 라인 없이 파일이 종료되었습니다.")
                        break
                    
                    next_line_content = next_line_raw.split('#', 1)[0].strip()
                    
                    if next_line_content.upper() == 'END':
                        print(f"[경고] %EXE_DEP: 'delt < 0' 이지만 추가 데이터 대신 END를 만났습니다.")
                        next_line_for_main_loop = None
                        break
                    
                    elif not next_line_content or next_line_content.startswith(('%', '/')):
                        print(f"[경고] %EXE_DEP: 'delt < 0' 이지만 필요한 추가 데이터 라인을 찾을 수 없습니다.")
                        next_line_for_main_loop = next_line_raw
                        
                    else: 
                        next_line_parts = next_line_content.split()
                        parsed_content['search_parameter'] = str(next_line_parts[0])
                        parsed_content['search_value'] = float(next_line_parts[1])
                        
                if itg > 0:
                    if next_line_for_main_loop is None:
                        next_line_raw = next(line_iterator, None)
                        
                        if next_line_raw is None:
                            print(f"[경고] %EXE_DEP: itg > 0 이나, 이후 라인 없이 파일이 종료되었습니다.")
                            break
                        
                        next_line_content = next_line_raw.split('#', 1)[0].strip()
                        
                        if next_line_content.upper() == 'END':
                            print(f"[경고] %EXE_DEP: 'itg > 0' 이지만 추가 데이터 대신 END를 만났습니다.")
                            next_line_for_main_loop = None
                            break
                        
                        elif not next_line_content or next_line_content.startswith(('%', '/')):
                            print(f"[경고] %EXE_DEP: 'itg > 0' 이지만 필요한 추가 데이터 라인을 찾을 수 없습니다.")
                            next_line_for_main_loop = next_line_raw
                            
                        else:
                            next_line_parts = next_line_content.split()
                            parsed_content['target_keff'] = float(next_line_parts[0])
                            parsed_content['target_boron'] = float(next_line_parts[1])
                break
            except (ValueError, IndexError) as e:
                print(f"[경고] %EXE_DEP 카드에서 잘못된 형식의 라인을 건너뜁니다: '{line_content}' (오류: {e})")
                continue
        return parsed_content, next_line_for_main_loop

MasterParser 메서드 테스트

In [6]:
parser = MasterParser()
base_file_path = "DEP_iSMR_DPL1_F01.INP"

deck = parser.parse_input_file(base_file_path)

In [7]:
deck.card_sequence

['JOB_TYP',
 'JOB_VER',
 'JOB_TIT',
 'JOB_IDE',
 'JOB_MDL',
 'GEN_MTH',
 'DEF_TFT',
 'GEN_THD',
 'GEN_PIN',
 'EDT_OUT',
 'GEN_DIM',
 'GEN_GEO',
 'GEN_SYM',
 'GEN_CDN',
 'LPD_BCH',
 'LPD_B&C',
 'LPD_C&X',
 'LPD_HFF',
 'ROD_CFG',
 'ROD_MAP',
 'EXE_STD',
 'EXE_DEP',
 'EXE_ROD',
 'EDT_OPT',
 '/',
 'EXE_CRS',
 'EXE_DEP',
 'EDT_OPT',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/']

In [8]:
deck.card_data

{'JOB_TYP': [{'raw_lines': ['        0       stead                                                           # irrst,jobtyp,\n',
    '        xsl     MAS_XSL\n',
    '        hff     MAS_HFF\n']}],
 'JOB_VER': [{'raw_lines': ['        4.0     2                                                               # xform,ichntyp\n']}],
 'JOB_TIT': [{'raw_lines': ['        Full Core, KNF iSMR Cycle 01, Critical Search\n']}],
 'JOB_IDE': [{'raw_lines': ['        DEPF    1                                                               # iplant,icycle\n']}],
 'JOB_MDL': [{'raw_lines': ['        2       5       0       1       0                                       # irod,iextc,ixsmod,iloc,idch,\n',
    '        0       5                                                               # iben,idete\n']}],
 'GEN_MTH': [{'raw_lines': ['        5       5       0       3       5                                       # imeth,itral,ibndc,ibupco,iasex,\n',
    '        5       5       5       5       5      

In [ ]:
class AstraParser():
    # 추후 구현

SyntaxError: incomplete input (1558064638.py, line 2)

## 3. DeckModifier
- MasterModifier(DeckModifier)
- AstraModifier(DeckModifier)

In [6]:
# --- 테스트를 위한 가상 클래스 정의 ---
class DeckModifier:
    def __init__(self, deck: 'Deck'):
        self.deck = deck

In [ ]:
class MasterModifier(DeckModifier):
    """
    TODO: 카드 이름을 입력받고, 카드 형식에 맞도록 내용을 수정할 수 있도록 메서드 구성 필요 (현재는 무작위 장전모형 생성에 특화된 메서드만 구현)
    
    
    """
    AXIAL_PATTERN_CONFIG: dict[str, dict] = {
        # num_compositions: 이 방식을 만드는 데 필요한 총 핵연료 조성의 개수.
        # has_unique_dominant: 가장 큰 비중(dominant)을 가진 영역에 배정된 조성(composition)들은 패턴 간 중복될 수 없음을 의미. 
            # True일 경우, 동일 composition이 다른 패턴의 dominant 영역에 쓰이면 중복으로 간주
        
        'uniform':             {'num_compositions': 1, 'has_unique_dominant': True},
        'highly_skewed':       {'num_compositions': 2, 'has_unique_dominant': True},
        'small_bothsides':     {'num_compositions': 2, 'has_unique_dominant': True},
        
        'moderately_skewed':   {'num_compositions': 2, 'has_unique_dominant': False},
        'balanced_2_regions':  {'num_compositions': 2, 'has_unique_dominant': False},
        'balanced_3_regions':  {'num_compositions': 3, 'has_unique_dominant': False},
        'near_2_domain':       {'num_compositions': 3, 'has_unique_dominant': False},
        'single_dominant':     {'num_compositions': 3, 'has_unique_dominant': False},
    }
    
    SYMMETRY_ALIASES = {
        # 사용자 입력 (Alias(별칭)) -> 내부 표준 이름 (Standard Name)
        "octant_symmetry": "rotate_octant",
        "quadrant_symmetry": "rotate_quadrant",
        "rotate_octant": "rotate_octant",     # 표준 이름 자체도 허용
        "rotate_quadrant": "rotate_quadrant", # 표준 이름 자체도 허용
        "none": "none",
        None: None
    }
    
    def __init__(self, deck: 'Deck'):
        super().__init__(deck)

    # ===================================================================
    # 1. 장전모형 생성 메서드
    # ===================================================================
    def generate_new_loading_pattern(self,
                                     axial_arrangement_patterns: list[str], # 몇개의 연료종류(fuel batch)를 사용할 것인지도 결정함
                                                                            # 예시: ['uniform', 'highly_skewed', 'balanced_2_regions'] 이면 3개
                                     available_compositions: list[str],
                                     lp_symmetry_type: str | None = None,
                                     )-> None:
        # TODO: 코드는 LPD_BCH를 이용하는, 1주기 노심에 대한 장전모형만 생성 가능, 셔플에 대한 기능 추후 구현 필요
        num_batch_types = len(axial_arrangement_patterns)
        print(f"[알림] 새로운 장전 패턴 생성: 핵연료 집합체 {num_batch_types}종, 대칭 조건='{lp_symmetry_type}', 축방향 배열 방식: {axial_arrangement_patterns}")

        # 1) 입력받은 패턴 개수만큼 정해진 조성을 할당하여 딕셔너리(batchID-composition)를 구성
            # 결과 예시: {'A1': {'compositions': ['C2'], 'pattern': 'uniform'}, 'A2': {'compositions': ['C4', 'C7'], ...}
        assigned_compositions  = self._assign_compositions_to_batches( 
            patterns=axial_arrangement_patterns,
            pool=available_compositions
        )
        
        # 2) 조성 목록을 구성하고 새로운 카드 작성, Deck 객체 업데이트
        new_lpd_c_and_x_card_data = self._generate_lpd_c_and_x_card(assigned_compositions)
            # 2a) _set_card_data(new_lpd_c_and_x_card_data): %LPD_C&X
        self._set_card_data(
            card_name='LPD_C&X',
            new_data=new_lpd_c_and_x_card_data
        )
            # 2b) _set_card_data(new_lpd_c_and_x_card_data): %LPD_HFF
                # 입력받은 데이터에서, 리플렉터 부분은 남겨두고 핵연료 부분만 교체
        self._set_card_data(
            card_name='LPD_HFF',
            new_data=new_lpd_c_and_x_card_data
        )
        
        # 3) 각 연료 배치에 할당된 조성 리스트를 패턴에 따라 axial node(26개(연료:24, 반사체:2))에 배치
            # %LPD_B&C 카드 구성 
        created_batches_data = self._generate_lpd_b_and_c_card( 
            assigned_compositions,
        )
        
        
        #print("created_batches_data \n", created_batches_data)
            # 3a) _set_card_data(): %LPD_B&C
        self._set_card_data(
            card_name='LPD_B&C',
            new_data=created_batches_data
        )
        
        # 4) 장전모형 구성 %LPD_BCH
            # 인자: LPD_B&C카드 데이터 / 대칭조건 / 풀코어 여부  
        created_lp_data = self._generate_lpd_bch_card(
            list(created_batches_data.keys()),
            symmetry_type = lp_symmetry_type, # 인자로 입력받은 lp_symmetry_type을 사용할 예정
            geometry_type=self.deck.card_data.get('GEN_SYM')[0]['geometry_type'], # FULL_CORE
            )

        self._set_card_data(
            card_name='LPD_BCH',
            new_data = created_lp_data
        )
        
        
        # 5) %GEN_DIM 카드 업데이트 (nbatch 및 ncomp 업데이트)
        # %GEN_DIM 카드의 nbatch를 수정하기 위함
        gen_dim_card_data = self.deck.card_data['GEN_DIM'][0]
        nbatch = len(created_batches_data) 
        ncomp = len(new_lpd_c_and_x_card_data)
        gen_dim_card_data['nbatch'] = nbatch
        gen_dim_card_data['ncomp'] = ncomp
        
        self._set_card_data(
            card_name='GEN_DIM',
            new_data = gen_dim_card_data
        )
        
        
    # ===================================================================
    # 2. 내부 구현 헬퍼 메서드
    # ===================================================================

    def _assign_compositions_to_batches(self, patterns: list[str], pool: list[str]) -> dict[str, list[str]]:
        """
        [단계 1] 입력받은 패턴에 따라 연료 배치들에 대해 모든 제약조건을 만족하는 조성을 할당
        
        - 제약 1 (특별 그룹 간 고유성): '특별 그룹'의 주(dominant) 조성은 `random.sample`을 통해 고유하게 선택됩니다.
        - 제약 2 (배치 내 고유성): 각 배치를 구성한 후 `set`을 이용해 내부 중복을 검사하고, 실패 시 재시도합니다.
        - 허용 규칙: 그 외 모든 조성(일반 그룹 주 조성, 모든 보조 조성)은 `random.choices`를 통해 중복을 허용하여 선택됩니다.
        """
        print(f"  └─ [알림] 핵종 조성 할당 시작...")
        if not pool:
            raise ValueError("[오류] 핵종 조성 풀('available_compositions')이 비어 있습니다.")

        # --- 1. 계획 (Planning): 필요한 각 역할의 총 개수 파악 ---
        num_special_dominant = sum(1 for m in patterns if self.AXIAL_PATTERN_CONFIG[m]['has_unique_dominant'])
        num_general_dominant = sum(1 for m in patterns if not self.AXIAL_PATTERN_CONFIG[m]['has_unique_dominant'])
        num_auxiliary = sum(self.AXIAL_PATTERN_CONFIG[m]['num_compositions'] - 1 for m in patterns)

        print(f"    ├─ 조성 할당 제약: UNIQUE DOMINANT COMP={num_special_dominant}개, COMMON DOMIANANT COMP={num_general_dominant}개, AUX COMP={num_auxiliary}개")

        max_retries = 100
        for attempt in range(max_retries):
            # --- 2. 캐스팅 (Casting): 역할에 맞는 조성을 '매번 새로' 뽑음 ---
            # [규칙] 특별 그룹의 주 조성은 반드시 고유해야 함 -> 비복원추출 (sample)
            if len(pool) < num_special_dominant: 
                raise ValueError(f"특별 주 조성({num_special_dominant}개)을 할당하기에 조성 풀({len(pool)}개)이 작습니다.")
            special_dominants_cast = random.sample(pool, k=num_special_dominant)
            
            # [규칙] 나머지 모든 역할은 중복 가능 -> 복원추출 (choices)
            num_other_roles = num_general_dominant + num_auxiliary
            other_roles_cast = random.choices(pool, k=num_other_roles)
            
            # 캐스팅된 결과 분리
            general_dominants_cast = other_roles_cast[:num_general_dominant]
            auxiliaries_cast = other_roles_cast[num_general_dominant:]

            # --- 3. 배역 부여 (Assignment) 및 유효성 검증 ---
            final_assignments: dict = {}
            is_valid_attempt = True
            
            # pop()을 사용하므로 복사본 생성
            temp_special_dominants = list(special_dominants_cast)
            temp_general_dominants = list(general_dominants_cast)
            temp_auxiliaries = list(auxiliaries_cast)

            for i, pattern in enumerate(patterns):
                batch_name = f'A{i+1}' # TODO: 1주기만 A로 시작하고, 2주기 이후의 경우 B, C 등으로 시작할텐데, 사용할 batch_group 이라는 인자('A' or 'B' 등..)를 따로 받아서 구성할 수 있도록 하자.
                config = self.AXIAL_PATTERN_CONFIG[pattern]
                
                # 주 조성(dominant) 할당
                if config['has_unique_dominant']:
                    dominant_comp = temp_special_dominants.pop(0)
                else:
                    dominant_comp = temp_general_dominants.pop(0)
                
                # 보조 조성(auxiliary) 할당
                num_aux_needed = config['num_compositions'] - 1
                aux_comps = [temp_auxiliaries.pop(0) for _ in range(num_aux_needed)]
                
                batch_compositions = [dominant_comp] + aux_comps
                
                # [규칙] 배치 내 조성은 반드시 고유해야 함 -> set으로 중복 검사
                if len(set(batch_compositions)) != len(batch_compositions):
                    # print(f"    ├─ [시도 {attempt+1:02d}] 배치 내 중복 발생! ({batch_name}: {batch_compositions}). 재시도...")
                    is_valid_attempt = False
                    break # 이 시도는 실패했으므로 다음 시도로 넘어감
                
                final_assignments[batch_name] = {
                    'compositions': batch_compositions,
                    'pattern': pattern
                }
                

            if is_valid_attempt:
                print(f"    └─ [시도 {attempt+1:02d}] 성공! 모든 제약 조건을 만족하는 할당을 찾았습니다.")
                for name, assignment_info  in final_assignments.items():
                    pattern_name = assignment_info['pattern']
                    comps = assignment_info['compositions']
                    print(f"        - {name} ({pattern_name}): {comps}")
                return final_assignments

        raise RuntimeError(f"최대 재시도 횟수({max_retries})를 초과했지만 유효한 조성 할당을 찾지 못했습니다.")



    def _generate_lpd_c_and_x_card(self, assignments: dict) -> dict[int, str]:
        """
        사용된 연료 조성들로 새로운 '조성-코드' 맵을 생성하고,
        self.deck에 있는 기존 반사체 조성 정보와 병합하여 최종 맵을 반환

        Args:
            assignments (dict): 배치별 조성 할당 정보.
                                {'A1': {'compositions': ['C2'], 'pattern': '...'}, ...}

        Returns:
            dict[int, str]: 반사체 정보와 새로 생성된 연료 조성 정보가 병합된,
                            'LPD_C&X' 카드에 사용될 최종 '코드-조성명' 맵.

        Raises:
            ValueError: self.deck에서 'LPD_C&X' 카드를 찾을 수 없을 경우.
        """
        
        # 1) assignments 딕셔너리에서 사용된 모든 고유한(unique) 연료 조성 이름을 추출
        all_compositions_list = []
        
        for assignment_info in assignments.values():
            all_compositions_list.extend(assignment_info['compositions']) # 현재 all_compositions_list: ['C3', 'C1', 'C1', 'C4', 'C3']
        
        unique_compositions_set = set(all_compositions_list)
        unique_fuel_comps = sorted(list(unique_compositions_set)) # 처리 후 : ['C1', 'C3', 'C4']
        
        # 2) 정렬된 연료 조성 리스트에 1부터 시작하는 순차적인 정수 코드를 부여
        fuel_compositions = {i + 1: comp_name for i, comp_name in enumerate(unique_fuel_comps)} # [{1: 'C1', 2:'C3', 3:'C4'}]
        print(f"  ├─ 연료 조성 인덱스: {len(fuel_compositions)} 종류")

        # 3) Deck 객체에서 기존 'LPD_C&X' 카드의 데이터를 로딩
        if not self.deck or not hasattr(self.deck, 'card_data'):
            raise AttributeError("[오류] Deck 객체 또는 'card_data' 속성이 존재하지 않습니다.")
        
        lpd_c_and_x_card_instances = self.deck.card_data.get('LPD_C&X')
        
        if not lpd_c_and_x_card_instances:
            raise ValueError("Deck 객체에서 'LPD_C&X' 카드를 찾을 수 없습니다. 반사체 정보를 가져올 수 없습니다.")

        # 4) 기존 맵에서 키가 음수인 반사체(Reflector) 정보만 추출
        base_card_data = lpd_c_and_x_card_instances[0]
        reflector_compositions = {comp_index: comp_name for comp_index, comp_name in base_card_data.items() if comp_index < 0}
        print(f"  ├─ 기존 반사체 조성 정보 보존: {len(reflector_compositions)} 종류")

        # 5. 보존된 반사체 맵과 새로 생성된 연료 맵을 병합하여 최종 맵을 완성합니다.
        new_card_data = reflector_compositions | fuel_compositions
        print(f"  └─ 최종 조성 맵 완성: 총 {len(new_card_data)} 종류 (반사체 + 연료)")
        print("    ├─ 카드 데이터 상세:")
        for idx, name in new_card_data.items():
            if idx < 0:
                print(f"    │ [반사체] {idx}: {name}")
            else:
                print(f"    │ [연료]   {idx}: {name}")
        print("    └─ 카드 데이터 출력 완료.")
        
        return new_card_data # 여기서 새로운 카드만 생성했고, 아직 Deck 객체를 업데이트 하진 않음


    def _generate_lpd_b_and_c_card(self, assignments: dict) -> dict:
        """
        [단계 2]: 각 배치당 할당된 조성과 패턴을 이용하여 연료 배치의 축방향 조성 분포(Axial Composition Distrubution) 리스트를 정의
        
        반사체 정보는 제외 (R로 시작하는 key는 그대로 유지)
        또는 반사체 정보만 추출하고, 새로운 연료배치별 축방향 조성 리스트와 합쳐서 new_data_card로 반환
        
        assignments는 batch_id를 key로 가지고 'composition'
        
        Examples:
            입력받은 인자 형태: {'A1': {'compositions': ['C2'], 'pattern': 'uniform'},
                                'A2': {'compositions': ['C4', 'C7'], 'pattern':'highly_skewed', 
                                ... }
                                
            LPD_C&X 데이터 형태(Deck 객체):  [{-1: 'REF_AXIAL',
                                                -2: 'REF_R1',
                                                -3: 'REF_R2',
                                                -4: 'REF_R3',
                                                -5: 'REF_R4',
                                                -6: 'REF_R5',
                                                -7: 'REF_R6',
                                                1: '400_516_000',
                                                2: '400_520_000',
                                                3: '400_720_N04',
                                                4: '400_724_704',
                                                5: '220_000_000'}]
            반환 형태: 반사체 + 연료에 대한 batch_id를 key값으로하고, axial composition list를 value로 가지는 딕셔너리
                    [{'R1': [-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2], 
                      'A1': [-1, 1, 1, 2, 2, 2, ..., -1]
                        ...}]
        
        """
        # 1) LPD_C&X 카드 내용을 Deck 객체로부터 가져옴
            # 해당 메서드를 호출하기전에 이미 수정되었다고 가정
        lpd_cx_card  = self.deck.card_data.get('LPD_C&X')
        if not lpd_cx_card :
            raise ValueError("[오류] Deck에서 'LPD_C&X' 카드를 찾을 수 없습니다.")
        
        # 2) LPD_C&X는 {식별자: 이름} 형식이므로, {이름: 식별자} 형식으로 역변환
            # 반사체가 아닌 연료 조성(index > 0)에 대해서만 생성 (반사체의 경우 -1: REF_AXIAL 등)
        
            # 리스트 안에 딕셔너리가 하나 들어있으므로, lpd_cx_card[0]을 통해 리스트 안의 딕셔너리에 접근
        compositions = lpd_cx_card[0]
        comp_name_to_codes = {}
        
        for comp_identifier, comp_name in compositions.items(): # .items()를 통해 키와 값 쌍을 꺼내옴
            # 반사체(comp_index < 0)이 아닌 연료 조성(comp_index > 0)일 경우 변환
            if comp_identifier > 0:
                comp_name_to_codes[comp_name] = comp_identifier
  
        # 3) 기존 Deck 객체의 LPD_B&C 카드에서 반사체('R'로 시작하는 배치) 정보를 보존
            # LPD_B&C 카드 데이터 조회
        base_lpd_bc_card = self.deck.card_data.get('LPD_B&C')
        if not base_lpd_bc_card:
           raise ValueError("[오류] Deck에서 'LPD_B&C' 카드를 찾을 수 없습니다.")

            # 반사체 ID만 걸러내어('R'로 시작하는 경우) 새로운 딕셔너리에 저장
        new_lpd_bc_data = {} # 새로 작성할 LPD_B&C 카드의 내용
        for batch_name, axial_distribution in base_lpd_bc_card[0].items():
            if batch_name.startswith('R'):
                new_lpd_bc_data[batch_name] = axial_distribution
            
        # 4) 입력으로 받은 할당된 연료배치(assignments)를 순회하며 새로운 축방향 분포 생성
        for batch_name, assignment_info in assignments.items(): # items()는 key, value 쌍을 반환받는 메서드
            pattern = assignment_info['pattern'] # e.g., 'pattern': 'uniform'
            comp_names = assignment_info['compositions'] # 여러 개, e.g., 'compositions': ['C4', 'C7']
        
            try:
                comp_unique_ids = [comp_name_to_codes[name] for name in comp_names] # e.g., ['C4', 'C7'] -> [3, 5] 등
            except KeyError as e: # key값인 name이 입력받은 assignments 딕셔너리에 존재하지 않으면 오류발생
                raise ValueError(f"'{batch_name} 배치의 조성(composition) '{e.args[0]}'이(가) LPD_C&X 카드에 정의되지 않았습니다.")

            # 입력받은 패턴과 각 batch_name 별 가지고있는 조성 고유번호 리스트(comp_unique_ids)를 이용해 axial_distribution 생성
            axial_distribution = self._construct_axial_distribution(pattern, comp_unique_ids) 
            
            # 생성된 분포를 최종 딕셔너리에 추가합니다.
            new_lpd_bc_data[batch_name] = axial_distribution

        return new_lpd_bc_data
    
    # _generate_lpd_b_and_c_card() 메서드에서 활용되는 헬퍼 함수
        # e.g., 'LPD_B&C': [{'R1': [-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2], ...]
    def _construct_axial_distribution(self,
                               pattern: str,
                               comp_unique_ids: list[int],
                               set_endpoints_to_minus_one: bool = True
                               ) -> list[int]:
        """
        - 입력된 패턴이름에 따라 축방향 분포 리스트를 생성
        - 핵연료 조성마다 부여된 고유번호 이용하여 24개의 축 방향 노드를 채운 후,
        - 양 끝에 반사체(-1)를 추가하여 26개 노드의 리스트를 반환 

        Args:
            pattern (str): 'uniform', 'highly_skewed' 등의 패턴 이름.
            comp_unique_ids (list[int]): 패턴에 사용될 조성 식별자(번호) 리스트,
                                           리스트의 첫 번째 요소가 주(dominant) 조성으로 간주.
            set_endpoints_to_minus_one (bool, optional):
                            - True (기본값): 24개 분포의 양 끝에 -1을 추가하여 26개 리스트를 반환
                            - False (TODO, 추후 구현): 26개 전체를 연료로 채우는 로직. 현재는 미구현 상태
        Returns:
            list[int]: 26개의 정수로 구성된 축 방향 분포 리스트.
            
        Raises:
            NotImplementedError: set_endpoints_to_minus_one이 False로 설정될 경우.
        
        """
        # --- 1. 입력 패턴 유효성 검증 ---
        if pattern not in self.AXIAL_PATTERN_CONFIG:
            raise ValueError(f"[오류] '{pattern}'은(는) 유효하지 않은 패턴 이름입니다.")
        
        config = self.AXIAL_PATTERN_CONFIG[pattern]
        required_comps = config['num_compositions']
        
        if len(comp_unique_ids) != required_comps:
            raise ValueError(f"패턴 '{pattern}'은(는) {required_comps}개의 조성 ID가 필요하지만, {len(comp_unique_ids)}개가 제공되었습니다.")
        
        # --- 2. 축방향 분포 리스트 생성 ---
        
        # 변수 초기화
        dominant_comp_id = comp_unique_ids[0]
        axial_distribution = []

        if set_endpoints_to_minus_one: # True인 경우
            NUM_FUEL_NODES = 24
        
            # --- 2a. 축방향 노드를 단일 영역으로 고려 (단일 조성) ---
            if pattern == 'uniform':
                # 정의: (24) -> 모든 노드를 주 조성 ID로 채움
                axial_distribution = [dominant_comp_id] * NUM_FUEL_NODES


            # --- 2b. 축방향 노드를 2개 영역으로 구분 (2개 조성) ---
            elif pattern == 'highly_skewed':
                # 정의: 작은 영역 < 15% (1~3개) -> (23,1), (22,2), (3, 20) 등
                aux_comp_id = comp_unique_ids[1]
                small_part_size = random.randint(1, 3) # 1~3 중에 선택
                
                if random.choice(['top', 'bottom']) == 'bottom': # 인덱스 0을 bottom으로 가정 (모든 시뮬레이션에서)
                    # [aux, aux, dom, dom, ...., dom] 형식으로 생성
                    axial_distribution = [aux_comp_id] * small_part_size + [dominant_comp_id] * (NUM_FUEL_NODES - small_part_size)
                else:
                    # [dom, dom, ..., aux] 형식으로 생성
                    axial_distribution = [dominant_comp_id] * (NUM_FUEL_NODES - small_part_size) + [aux_comp_id] * small_part_size

            elif pattern == 'moderately_skewed':
                # 정의: 작은 영역 15%~40% (4~9개) -> (20, 4), (9, 15) 등
                aux_comp_id = comp_unique_ids[1]
                small_part_size = random.randint(4, 9)
                if random.choice(['top', 'bottom']) == 'bottom':
                    axial_distribution = [aux_comp_id] * small_part_size + [dominant_comp_id] * (NUM_FUEL_NODES - small_part_size)
                else:
                    axial_distribution = [dominant_comp_id] * (NUM_FUEL_NODES - small_part_size) + [aux_comp_id] * small_part_size
            
            elif pattern == 'balanced_2_regions':
                # 정의: 작은 영역 40% 초과, (10~12개) -> (14,10), (13,11), (12,12) 등
                aux_comp_id = comp_unique_ids[1]
                part_a_size = random.randint(10, 12)
                part_b_size = NUM_FUEL_NODES - part_a_size

                if random.choice([True, False]):
                    axial_distribution = [dominant_comp_id] * part_b_size + [aux_comp_id] * part_a_size
                else:
                    axial_distribution = [aux_comp_id] * part_a_size + [dominant_comp_id] * part_b_size
            
            # NOTE: small_bothsides의 경우 축방향 노드는 3개로 구분되나 조성은 2개 
    
            elif pattern == 'small_bothsides':
                # 정의: (1, 22, 1), (2, 20, 2) 등, ends_size는 2 이하 -> 양 끝이 작은 대칭 형태
                 # (S, L, S) 형태이고, S ∈ {1, 2}
                aux_comp_id = comp_unique_ids[1]
                ends_size = random.randint(1, 2)
                
                center_size = NUM_FUEL_NODES - (ends_size * 2)
                axial_distribution = [aux_comp_id] * ends_size + [dominant_comp_id] * center_size + [aux_comp_id] * ends_size

            # --- 2c. 축방향 노드를 3개 영역으로 구분 (3개 조성) ---
            # 파트 L > M > S 
            elif pattern == 'balanced_3_regions':
                # 정의: L - S ≤ 4 -> (8,8,8), (9,8,7), (10,8,6) 등
                comp_id1, comp_id2, comp_id3 = comp_unique_ids[0], comp_unique_ids[1], comp_unique_ids[2]
                # 가능한 조합을 사전에 정의
                valid_splits = [(8, 8, 8), (9, 8, 7), (10, 8, 6), (10, 7, 7), (9, 9, 6)]
                # 조합을 선정
                sizes = random.choice(valid_splits)
                part1 = [comp_id1] * sizes[0]
                part2 = [comp_id2] * sizes[1]
                part3 = [comp_id3] * sizes[2]
                # valid_split에 정의된 크기대로 구성
                parts = [part1, part2, part3] 
                random.shuffle(parts) # shuffle을 통해서 parts 리스트의 요소 순서를 무작위로 변경
                
                axial_distribution = parts[0] + parts[1] + parts[2]
                
                
            elif pattern == 'near_2_domain':
                # 정의: 큰 덩어리에서 1~2개 파편화. 파편은 양 끝에만 위치.
                # 로직: 2개 영역으로 나눈 뒤, 큰 영역에서 '파편'을 떼어내고,
                #       이 파편을 전체 분포의 맨 앞 또는 맨 뒤에 배치
                comp_id1, comp_id2, comp_id3 = comp_unique_ids[0], comp_unique_ids[1], comp_unique_ids[2]

                # 1. 24개 노드를 두 개의 덩어리로 나눔
                part_a_size = random.randint(4, 12)
                part_b_size = NUM_FUEL_NODES - part_a_size

                larger_part_size = max(part_a_size, part_b_size)
                smaller_part_size = min(part_a_size, part_b_size)

                # 2. 더 큰 덩어리에서 1~2개 크기의 "파편(chip)"을 추출
                chip_size = random.randint(1, 2)
                final_large_part_size = larger_part_size - chip_size

                # 3. 3개의 파트에 조성 ID를 무작위로 할당
                shuffled_comp_ids = random.sample([comp_id1, comp_id2, comp_id3], k=3)
                
                part_final_large = [shuffled_comp_ids[0]] * final_large_part_size
                part_smaller = [shuffled_comp_ids[1]] * smaller_part_size
                part_chip = [shuffled_comp_ids[2]] * chip_size

                # 4. 파편(chip)을 제외한 나머지 두 개의 큰 덩어리를 먼저 합치고 무작위 순서로 구성
                main_parts = [part_final_large, part_smaller]
                random.shuffle(main_parts)
                final_main_parts = main_parts[0] + main_parts[1]

                # 5. 파편을 전체 분포의 맨 앞(bottom) 또는 맨 뒤(top)에 할당
                if random.choice(['top', 'bottom']) == 'bottom':
                    # 파편을 맨 앞(아래)에 붙임 (Bottom, index 0)
                    axial_distribution = part_chip + final_main_parts
                else:
                    # 파편을 맨 뒤(위)에 붙임 (Top, last index)
                    axial_distribution = final_main_parts + part_chip
                
                
            elif pattern == 'single_dominant':
                # 정의: 가장 작은파편 S ≥ 3 이고 L ≥ 1.5 * M
                # 제약조건을 만족하는 (L, M, S) 조합 중 하나를 무작위로 선택
                comp_id1, comp_id2, comp_id3 = comp_unique_ids[0], comp_unique_ids[1], comp_unique_ids[2]

                # 모든 제약조건을 만족하는 유효한 (L, M, S) 조합 목록
                valid_splits = [
                    (18, 3, 3), (17, 4, 3), (16, 5, 3), (16, 4, 4), (15, 6, 3),
                    (15, 5, 4), (14, 7, 3), (14, 6, 4), (14, 5, 5), (13, 8, 3),
                    (13, 7, 4), (13, 6, 5), (12, 8, 4), (12, 7, 5), (12, 6, 6)
                ]
                
                # 유효한 조합 중 하나를 무작위로 선택
                l_size, m_size, s_size = random.choice(valid_splits)

                # 3개의 덩어리 크기에 3개의 조성 ID를 무작위로 할당
                shuffled_comp_ids = random.sample([comp_id1, comp_id2, comp_id3], k=3)
                parts = [
                    [shuffled_comp_ids[0]] * l_size,
                    [shuffled_comp_ids[1]] * m_size,
                    [shuffled_comp_ids[2]] * s_size
                ]
                
                # 파트를 합친 후 무작위로 섞어 축방향 분포 리스트를 구성
                random.shuffle(parts)
                axial_distribution = [item for part in parts for item in part]
                
            else:
                print(f"[경고] '{pattern}' 패턴 로직 미구현. 임시로 uniform 처리.")
                axial_distribution = [dominant_comp_id] * num_fuel_nodes
            
            return [-1] + axial_distribution + [-1]
        
        # --- set_endpoints_to_minus_one == False인 경우 ---
        else: 
            # TODO: 26개 노드 전체를 연료로 채우는 로직 구현 필요
            num_fuel_nodes = 26
            raise NotImplementedError(
                "[오류] set_endpoints_to_minus_one=False 모드 (26개 전체 연료 분포)는 아직 구현되지 않았습니다."
            )


    def _generate_lpd_bch_card(self,
                               batch_ids: list[str],
                               symmetry_type: str | None = None,
                               geometry_type = "FULL_CORE",
                               keep_current_reflectors_fixed = True,
                               ) -> dict:
        """
        [단계 3] LPD_BCH 카드 생성
        - NOTE: 베이스 입력덱(파일)에서 작성된 장전모형의 크기를 그대로 사용
        - generate_new_loading_pattern 에서 입력받는 pattern 개수만큼 연료 배치가 결정됨 (A1, A2, ..., An)
        - 해당 메서드에서 중요한건, FULL_CORE/ QUATER_CORE 여부 및 symmetry_type임

        Args:
            batch_ids (list[str]): 새로 작성된 LPD_B&C 카드의 연료 배치(fuel batch id) 목록
                # list(created_batches_data.keys())를 이용해서 가져옴
            symmetry_type (str | None): 대칭 옵션 인자이며 기본은 None, 가능 옵션은 'rotate_quadrant' 또는 'rotate_octant'
            geometry_type (str): 노심 형상 타입, 기본은 'FULL_CORE'이며, 사용가능한 타입은 메서드 내부에 목록 구성
            keep_current_reflectors_fixed (bool): 장전모형 내부의 반사체 위치 고정 여부, 기본값은 True
            
        Returns:
            new_lpd_bch_card (dict[list[list]]): n x n 장전모형 행렬 
        
        Examples:
         'LPD_BCH': [{'loading_pattern': 
                       [['o',  'o',  'R4', 'R2', 'R2', 'R1', 'R2', 'R2',  R4', 'o',  'o'],
                        ['o',  'R6', 'R3', 'A1', 'A2', 'A2', 'A2', 'A1', 'R3', 'R6', 'o'],
                        ['R4', 'R3', 'A1', 'A2', 'A4', 'A2', 'A4', 'A2', 'A1', 'R3', 'R4'],
                        ['R2', 'A1', 'A2', 'A4', 'A4', 'A4', 'A4', 'A4', 'A2', 'A1', 'R2'],
                        ['R2', 'A2', 'A4', 'A4', 'A5', 'A5', 'A5', 'A4', 'A4', 'A2', 'R2'],
                        ['R1', 'A2', 'A2', 'A4', 'A5', 'A4', 'A5', 'A4', 'A2', 'A2', 'R1'],
                        ['R2', 'A2', 'A4', 'A4', 'A5', 'A5', 'A5', 'A4', 'A4', 'A2', 'R2'],
                        ['R2', 'A1', 'A2', 'A4', 'A4', 'A4', 'A4', 'A4', 'A2', 'A1', 'R2'],
                        ['R4', 'R3', 'A1', 'A2', 'A4', 'A2', 'A4', 'A2', 'A1', 'R3', 'R4'],
                        ['o',  'R6', 'R3', 'A1', 'A2', 'A2', 'A2', 'A1', 'R3', 'R6', 'o'],
                        ['o',  'o',  'R4', 'R2', 'R2', 'R1', 'R2', 'R2', 'R4', 'o',  'o']]
                    }],
        
        """
        # 반사체 고정 여부 확인
        if not keep_current_reflectors_fixed:
            # TODO: keep_current_reflectors_fixed=False 로직 구현
            raise NotImplementedError("[오류] keep_current_reflectors_fixed=False 옵션은 아직 구현되지 않았습니다.")
    
        # symmetry_type 처리
        input_symmetry_key = symmetry_type.lower() if isinstance(symmetry_type, str) else symmetry_type 
        standardized_symmetry_name = self.SYMMETRY_ALIASES.get(input_symmetry_key)
        CANDIDATE_SYMMETRY_TYPES = [
            'rotate_quadrant',
            'rotate_octant',
            'none',
            None
        ]
        if standardized_symmetry_name not in CANDIDATE_SYMMETRY_TYPES:
            raise ValueError(
                f"[오류] 지원하지 않는 대칭 옵션입니다: '{symmetry_type}'\n"
                f"[알림] 가능한 대칭 옵션: {', '.join(k for k in self.SYMMETRY_ALIASES if k is not None)}\n"
        )
        
        CANDIDATE_GEOMETRY_TYPES = [
            'FULL_CORE',
            'QUATER_CORE',
            'OCTANT_CORE',
            'UNKNOWN'
        ]
        if geometry_type not in CANDIDATE_GEOMETRY_TYPES:
            raise ValueError(
                f"[오류] 지원하지 않는 노심 형상입니다: {geometry_type}\n"
                f"[알림] 가능한 형상 타입: {', '.join(CANDIDATE_GEOMETRY_TYPES)}\n"
                f"[알림] OCTANT CORE의 경우 아직 구현되지 않았습니다."
            )
        
        geometry_type = geometry_type.upper()

        # --- 1. 배치 ID (batch ID) 분류 ---
        all_batch_ids = batch_ids # 리스트 형태이며 LPD_B&C에 정의된 모든 Batch 이름을 가져옴
        reflector_batch_ids = [] # 'R'로 시작하는 반사체의 batch id 리스트
        fuel_batch_ids = [] # 'A','B', ... 등으로 시작하는 연료의 batch id 리스트
        EXPECTED_FUEL_BATCH_ID_LETTERS = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H')
        
        for batch_id in all_batch_ids:
            if not batch_id:
                print("[경고] 비어있는 배치(batch) 이름이 발견되었습니다.")
                continue
            if batch_id.startswith('R'):
                reflector_batch_ids.append(batch_id)
            elif batch_id.startswith(EXPECTED_FUEL_BATCH_ID_LETTERS):
                fuel_batch_ids.append(batch_id)
            else:
                print(f"[경고] 연료 배치 이름 '{batch_id}'이(가) 예상된 형식('{'/'.join(EXPECTED_FUEL_BATCH_ID_LETTERS)}'로 시작)이 아닙니다.")
                fuel_batch_ids.append(batch_id)

        # --- 2. 노심 장전모형 형상 타입 (geometry_type) 확인  ---
        if geometry_type == 'SAME':
            try:
                geometry_type = self.deck.card_data['GEN_SYM'][0]['geometry_type']
            except (KeyError, IndexError):
                raise ValueError("[오류] 'same' 옵션 사용 시 'GEN_SYM' 카드에서 geometry_type을 찾을 수 없습니다.")

        if geometry_type == 'OCTANT_CORE':
            raise NotImplementedError("[오류] OCTANT_CORE의 경우 아직 구현되지 않았습니다.")
        if geometry_type == 'UNKNOWN':
            raise ValueError("[오류] 노심 형상 타입이 UNKNOWN 이므로 장전모형을 생성할 수 없습니다.")

        
        # --- 3. 장전모형 행렬 구성 ---
        
        # keep_current_reflectors_fixed == False이면
            # TODO(미구현), 우선 R로 시작하는 위치는 기록해두고, 위치에 해당하는 Reflector를 임의로 배치하도록 구성
        
        # 3a. 기존 장전모형(base_loading_pattern)과 그 크기(base_lp_size)를 확인
        try:
            base_loading_pattern = self.deck.card_data['LPD_BCH'][0]['loading_pattern']
            base_lp_size = len(base_loading_pattern)
        except (KeyError, IndexError, TypeError): # card_data['LPD_BCH'][0]['loading_pattern'] 이므로 각각 하나라도 잘못되면 오류 포착
            raise ValueError("[오류] 'LPD_BCH' 카드에서 기존 장전모형을 찾을 수 없습니다.")
        
        # 3b. base_loading_pattern을 확인하여 첫 모서리칸([0][0])이 'o'인지의 여부로 geometry_type을 추론     
        base_geometry_type = 'FULL_CORE' if base_loading_pattern[0][0] == 'o' else 'QUATER_CORE'
        target_geometry_type = geometry_type
        
        full_core_size: int
        quarter_core_size: int

        if base_geometry_type == 'FULL_CORE':
            full_core_size = base_lp_size
            quarter_core_size = (base_lp_size // 2) + 1
        else: # base_geometry_type == 'QUATER_CORE'
            quarter_core_size = base_lp_size
            full_core_size = (base_lp_size * 2) - 1
    
        print(f"    ├─ 노심 크기 정의: FULL_CORE={full_core_size}x{full_core_size}, QUATER_CORE={quarter_core_size}x{quarter_core_size}")
         
         
        # 3c. 목표 형상에 맞는 '작업용 장전모형(temp_loading_pattern)'을 생성
            # 1) FULL_CORE CASE
            # 목표형상이 FULL_CORE이고, base_geometry_type이 FULL_CORE로 동일하면서 symmetry 조건이 None인 경우 -> 그대로 복사
                # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 '연료 위치'임
                # 연료 배치 후(에는 장전모형 대칭,복사,변경 등은 없음
            # 목표형상이 FULL_CORE이고, base_geometry_type이 FULL_CORE로 동일하나 symmetry 조건이 rotate_quadrant인 경우 -> 4분면만 복사(크기는 QUATER_CORE)
                # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 연료 위치임
                # fuel_batch_ids를 이용해 조건에 맞게 연료 배치 후에, roate_quadrant이므로 회전대칭 적용 (QUATER_CORE -> FULL_CORE)
            # 목표형상이 FULL_CORE이고, base_geometry_type이 FULL_CORE로 동일하나 symmetry 조건이 rotate_octant인 경우 -> 4분면만 복사(크기는 QUATER_CORE)
                # '연료 위치'/ 반사체 위치를 확인한 뒤, '연료 배치 가능 영역'은 QUATER_CORE의 주 대각선을 기준으로 상 삼각행렬 (즉, 연료 위치들 중 대각대칭이 되도록 하는 행렬 영역)
                # fuel_batch_ids를 이용해 조건에 맞게 상삼각행렬에 배치 후에 대각대칭으로 QUATER_CORE를 완성하고 이후 회전대칭 적용하여 FULL_CORE 작성
                
            # 2) QUATER_CORE CASE
            # ** QUATER_CORE이면서 symmetry 조건이 none일때는 rotate_quadrant로 취급
            # 목표형상이 QUATER_CORE이고, base_geometry_type이 QUATER_CORE로 동일하면서 symmetry 조건이 rotate_quadrant인 경우 -> 그대로 복사
                # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 연료 위치임
                # fuel_batch_ids이용해 연료 배치, 회전 대칭 적용 필요 없이 그대로 QUATER_CORE이고 유지
            # 목표형상이 QUATER_CORE이고, base_geometry_type이 QUATER_CORE로 동일하면서 symmetry 조건이 rotate_octant인 경우 
                # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 QUATER_CORE의 주 대각선을 기준으로 상 삼각행렬 (즉, 연료 위치들 중 대각대칭이 되도록 하는 행렬 영역)
                # fuel_batch_ids를 이용해 조건에 맞게 상삼각행렬에 배치 후에 대각대칭으로 QUATER_CORE를 완성
            
            # 3) FULL_CORE -> QUATER_CORE CASE
            # 목표형상이 QUATER_CORE이고, base_geometry_type이 FULL_CORE일 경우 4사분면(우하단)을 기준으로 축소해야함
                # 축소한 QUATER_CORE 기준
                # symmetry 조건이 rotate_octant인 경우 
                    # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 QUATER_CORE의 주 대각선을 기준으로 상 삼각행렬 (즉, 연료 위치들 중 대각대칭이 되도록 하는 행렬 영역)
                # symmetry 조건이 rotate_quadrant인 경우
                    # '연료 위치'/ 반사체 위치 확인한 뒤, '연료 배치 가능 영역'은 연료 위치임
    
        
        # 3c. 작업용 장전모형(base_lp_for_work) 생성
        if target_geometry_type == 'QUATER_CORE' and standardized_symmetry_name == None:
            standardized_symmetry_name = 'rotate_quadrant'

        # --- 3c. 작업용 장전모형(base_lp_for_work) 생성 ---
        # 여기서 geometry_type에 상관 없이, symmetry_type이 none이 아니라면 무조건 QUATER_CORE 형태로 base_lp_for_work을 이용

        base_lp_for_work = [] # base_loading_pattern(원본)은 그대로 두고, 임시 행렬 base_lp_for_work는 작업에 맞게 축소/가공할 목적으로 이용됨
        if target_geometry_type == 'FULL_CORE' and standardized_symmetry_name == None:
            # 대칭 조건 없이 FULL_CORE를 생성할 때는, base도 반드시 FULL_CORE여야 함
            if base_geometry_type != 'FULL_CORE':
                raise ValueError(f"[오류] 대칭 조건 없이 FULL_CORE를 생성하려면 기본 장전모형도 FULL_CORE여야 합니다.")
            base_lp_for_work = base_loading_pattern # 그대로 복사
        else: # 즉, symmetry_type이 'rotate_quadrant' 또는 'rotate_octant'인 경우
            # 1) base가 FULL_CORE이면, 4사분면(우하단)을 추출하여 작업 단위로 삼음
            if base_geometry_type == 'FULL_CORE':
                # 예: base_loading_pattern의 크기가 11x11이면 quarter_core_size = (11 // 2) + 1 = 6
                # start_idx = 11 - 6 = 5
                # 즉, 5번째 행/열부터 4사분면(우하단) 추출 (base_loading_pattern[5:])
                # 크기가 11이면 [0] ~ [10]의 인덱스가 존재하므로, [0, 1, 2, 3, 4, / 5 / 6, 7, 8, 9, 10] 중에서 5번째부터 인덱스를 추출  
                start_idx = full_core_size - quarter_core_size
                base_lp_for_work = [row[start_idx:] for row in base_loading_pattern[start_idx:]]
            
            # 2) base가 QUATER_CORE이면 그대로 사용
            else:
                base_lp_for_work = base_loading_pattern
        
        working_lp_size = len(base_lp_for_work)
        temp_loading_pattern = copy.deepcopy(base_lp_for_work)
        fuel_positions = []

        # 연료 배치 가능 영역을 판별하는 헬퍼 함수
            # - 입력된 batch_id가 문자열(str)이고, 'R'로 시작하지 않으며(반사체가 아님),'o'가 아닌 경우(비어있는 영역이 아님),
            # => 연료 배치가 가능한 위치로 간주하여 True 반환
        def is_fuel(batch_id):
            if not isinstance(batch_id, str): return False
            return not batch_id.startswith('R') and batch_id != 'o'

        # 연료 배치 가능 위치(fuel_positions) 찾기
            # temp_loading_pattern은 연료 배치가 가능한 공간만 None으로 비워진, 장전모형의 '틀'의 역할을 함
        if standardized_symmetry_name == 'rotate_quadrant':
            for r in range(working_lp_size):
                for c in range(working_lp_size):
                    if is_fuel(base_lp_for_work[r][c]):
                        fuel_positions.append((r, c)) # fuel_positions: 연료집합체가 위치한 좌표쌍을 저장하는 리스트
                        temp_loading_pattern[r][c] = None # temp_loading_pattern은 실제 무작위 batch_id를 넣기 위한 빈 공간.
                                                          # 연료 배치가 가능한 위치는 None으로 표시하여 이후 무작위 배치가 가능하도록 함.
                                                          # 나머지 공간(비어있거나 반사체 위치)은 deepcopy로 인해 'o' 또는 'R'로 시작하는 값이 이미 들어 있음.
        elif standardized_symmetry_name == 'rotate_octant':
            for r in range(working_lp_size):
                for c in range(working_lp_size):
                    if is_fuel(base_lp_for_work[r][c]):
                        if c >= r: # rotate_quadrant 대부분 로직이 동일하지만, QUATER_CORE의 주 대각선을 기준으로 대칭조건 이용
                            fuel_positions.append((r, c))  # 연료 집합체 위치가 넣어져야할 상삼각 영역의 좌표쌍을 저장
                        temp_loading_pattern[r][c] = None # 연료 위치는 모두 None으로 바꿈 (상삼각/하삼각 모두)
        else: # standardized_symmetry_name == None인 경우, Full Core이므로
            # 연료 배치 가능 위치를 모두 찾음
            for r in range(working_lp_size):
                for c in range(working_lp_size):
                    if is_fuel(base_lp_for_work[r][c]):
                        fuel_positions.append((r, c))
                        temp_loading_pattern[r][c] = None

        # --- 4. 연료 배치 ---
        # temp_loading_pattern은 연료 배치가 가능한 위치를 None으로 표시한 상태
        # fuel_positions는 연료 배치가 가능한 위치의 좌표쌍을 저장한 리스트
        # fuel_batch_ids는 연료 배치에 사용할 배치 ID의 리스트
        num_fuel_positions = len(fuel_positions)

        if not fuel_batch_ids:
            raise ValueError("[오류] 배치 가능한 연료(fuel_batch_ids)가 비어 있어 장전모형을 채울 수 없습니다.")

        if num_fuel_positions == 0:
            print("[경고] 장전모형 내에 연료를 배치할 위치가 없습니다.")
            # 연료를 배치할 위치가 없으면, temp_loading_pattern은 이미 올바르게 복사되어 있으므로
            # 추가적인 연료 배치 없이 다음 단계로 진행됩니다.

        # [작동 과정 설명]
        # 1. standardized_symmetry_name에 따라 연료 배치 최소 개수(min_placements_required)를 결정
        #    - rotate_octant: 최소 1개
        #    - rotate_quadrant: 전체 연료 위치의 10% (최소 2개)
        #    - Full core & 대칭 없음: 전체 연료 위치의 20% (최소 5개)
        # 2. min_placements_required가 전체 연료 위치 개수를 초과하지 않도록 보정
        # 3. 연료 위치 리스트(fuel_positions)를 무작위로 섞어, 배치 순서를 랜덤화
        # 4. 각 연료 위치(r, c)에 대해 fuel_batch_ids에서 무작위로 하나를 선택하여 배치

        # [예시]
        # fuel_positions = [(0,1), (0,2), (1,1), ...]  # 연료를 배치할 좌표 리스트
        # fuel_batch_ids = ['A1', 'A2', 'A3']         # 배치 가능한 연료 집합체 ID
        # temp_loading_pattern[r][c] = random.choice(fuel_batch_ids)
        #   -> (0,1) 위치에 'A2', (0,2) 위치에 'A1', (1,1) 위치에 'A3' 등 랜덤 배치

        min_placements_required = 0
        if standardized_symmetry_name == 'rotate_octant':
            min_placements_required = 1 # 최소 1개는 무작위로 뽑아 배치
        elif standardized_symmetry_name == 'rotate_quadrant':
            # 전체 가능 영역의 10% (최소 2개)
            min_placements_required = max(2, int(num_fuel_positions * 0.10))
        elif standardized_symmetry_name is None and target_geometry_type == 'FULL_CORE':
            # Full core, 대칭 없음: 20% (최소 5개)
            min_placements_required = max(5, int(num_fuel_positions * 0.20))
        
        # min_placements_required가 전체 위치 개수를 초과하지 않도록 보정
        min_placements_required = min(min_placements_required, num_fuel_positions)

        # 연료 위치 리스트를 무작위로 섞어, 어떤 물리적 위치에 어떤 순서로 연료가 배치될지 결정
        random.shuffle(fuel_positions)

        # 모든 연료 위치에 대해 fuel_batch_ids 풀에서 무작위로 하나의 batch_id를 뽑아 배치
        for r, c in fuel_positions:
            temp_loading_pattern[r][c] = random.choice(fuel_batch_ids)
        

        # --- 5. 대칭 적용 및 최종 장전모형 생성 ---
            # NOTE: 회전대칭을 적용해야 하지만, 현재는 반사대칭으로 구현되어 있음
        if standardized_symmetry_name == 'rotate_octant':
            for r in range(working_lp_size):
                for c in range(r): # 반복문 자체가 r > c라는 조건
                    # (r, c)가 하삼각행렬(대각 제외)에 해당하는 경우,
                    # 상삼각행렬의 (c, r) 위치 값을 복사하여 대칭을 맞춤
                    temp_loading_pattern[r][c] = temp_loading_pattern[c][r]
        
        final_loading_pattern = []
        if target_geometry_type == 'FULL_CORE': # FULL_CORE일때 symmetry_type 이 없다면 그대로 복사
            if standardized_symmetry_name is None:
                # symmetry_type이 None인 경우(대칭 조건 없이 FULL_CORE 생성):
                # temp_loading_pattern(임시 작업 행렬)을 그대로 최종 장전모형(final_loading_pattern)으로 사용
                final_loading_pattern = temp_loading_pattern
            else: # FULL_CORE이면서 symmetry 조건이 존재한다면
                # symmetry_type이 rotate_quadrant 또는 rotate_octant인 경우:
                    # 4사분면(우하단)을 기준으로 회전 대칭을 적용하기 위해
                    # full_core_size(f_size) x full_core_size(f_size) 크기의 빈 행렬을 생성
                    # 이후 4사분면 데이터를 대칭 복사하여 전체 FULL_CORE 행렬을 완성
                q_size = quarter_core_size
                f_size = full_core_size
                final_loading_pattern = [[None for _ in range(f_size)] for _ in range(f_size)] # 빈 행렬 구성
                
                # 4사분면(우하단)의 temp_loading_pattern을 중심(0,0) 기준으로 90도씩 회전하여 FULL_CORE의 각 사분면에 채움
                # 중심 기준 회전: temp_loading_pattern[0][0]이 FULL_CORE의 각 사분면의 중심이 되도록 배치
                # 1. 4사분면(우하단) 복사
                start_idx = f_size - q_size
                for r in range(q_size):
                    for c in range(q_size):
                        final_loading_pattern[start_idx + r][start_idx + c] = temp_loading_pattern[r][c]

                # 2. 중앙 열 기준 y축 대칭 (3사분면: 좌하단)
                for r in range(start_idx, f_size):
                    for c in range(start_idx, f_size):
                        mirror_c = f_size - 1 - c
                        final_loading_pattern[r][mirror_c] = final_loading_pattern[r][c]

                # 3. 중앙 행 기준 x축 대칭 (1, 2사분면: 상단)
                for r in range(start_idx, f_size):
                    mirror_r = f_size - 1 - r
                    for c in range(f_size):
                        final_loading_pattern[mirror_r][c] = final_loading_pattern[r][c]

                # 예시: temp_loading_pattern[0][0]이 각 사분면의 중심에 위치하도록 회전 복사됨
        elif target_geometry_type == 'QUATER_CORE':
            final_loading_pattern = temp_loading_pattern
            
        return {'loading_pattern': final_loading_pattern}
        

    
    # 카드 업데이트 메서드
    def _set_card_data(self, card_name: str, new_data: dict, card_index: int = 0) -> None:
        # TODO: LPD_BCH를 수정해야 할 때 해당 메서드도 수정 가능성이 존재함
        """
        Deck 객체 내에서 지정된 카드의 데이터를 new_data를 기반으로 설정합니다.
        카드 이름에 따라 특별한 처리 로직을 적용합니다.

        [동작 방식]
        - 'LPD_C&X': new_data를 그대로 사용하여 데이터를 완전히 교체합니다.
        - 'LPD_HFF': new_data에서 연료 정보(키 > 0)만 추출하여 데이터를 교체합니다.
        - 그 외 모든 카드: new_data를 그대로 사용하여 데이터를 완전히 교체합니다.

        Args:
            card_name (str): 데이터를 설정할 카드의 이름 (self.deck.card_data의 키).
            new_data (dict): 설정의 기반이 될 새로운 카드 데이터 (반사체+연료 포함 가능).
            card_index (int, optional): 교체할 인스턴스의 순서(0부터 시작). 기본값 0.

        Raises:
            KeyError: self.deck.card_data에 해당 card_name이 존재하지 않을 경우.
            IndexError: 지정한 card_index가 유효한 범위를 벗어날 경우.
        """
        # 1. 공통 유효성 검사
        if card_name not in self.deck.card_data:
            raise KeyError(f"Deck에 '{card_name}' 카드가 존재하지 않습니다.")
        
        card_instances = self.deck.card_data[card_name]
        
        if not (0 <= card_index < len(card_instances)):
            raise IndexError(f"'{card_name}' 카드의 인덱스({card_index})가 유효한 범위를 벗어났습니다.")

        final_card_value = {}

        # 2. 카드 이름에 따라 로직을 분기합니다.
        if card_name == 'LPD_HFF':
            # [특수 동작: LPD_HFF]
            # new_data에서 반사체 정보를 제외하고, 연료 정보(양수 키)만 필터링합니다.
            print(f"  ├─ '{card_name}' 카드의 데이터를 [반사체를 제외한 연료 조성(composition)만 교체]합니다.")
            
            fuel_only_map = {code: name for code, name in new_data.items() if code > 0}
            final_card_value = fuel_only_map
            
        else:
            # [일반 동작: LPD_C&X 및 그 외 모든 카드]
            # new_data를 그대로 사용하여 데이터를 완전히 교체합니다.
            print(f"  ├─ '{card_name}' 카드의 데이터를 [새로운 데이터로 교체]합니다.")
            final_card_value = new_data

        # 3. 분기 로직을 통해 결정된 최종 값으로 self.deck의 데이터를 업데이트합니다.
        card_instances[card_index] = final_card_value
        print(f"  └─ '{card_name}' 카드 업데이트 완료.")



# --- 테스트용 ---
if __name__ == '__main__':
    
    modifier = MasterModifier(deck)
    

    comp_pool = [
        '220_000_000', '400_516_000', '400_520_000', '400_720_N04',
        '400_724_704', '495_000_N20', '495_520_508', '495_520_512',
        '495_524_504', '495_720_716', '495_724_712'
    ]
    
    
    # 생성할 축 방향 배열 방식 목록
    # patterns_to_generate = [
    #     'uniform',             # 특별 그룹
    #     'highly_skewed',       # 특별 그룹
    #     'small_bothsides',     # 특별 그룹
    #     'moderately_skewed',   # 일반 그룹
    #     'balanced_3_regions',  # 일반 그룹
    #     'single_dominant',     # 일반 그룹
    #     'near_2_domain',
    #     'balanced_2_regions'
    # ]
    patterns_to_generate = ['uniform', 'highly_skewed', 'highly_skewed', 'uniform']
    
    print("="*50)
    print("테스트 실행 시작")
    print(f"조성 풀: {comp_pool}")
    print("="*50)
    
    try:
        modifier.generate_new_loading_pattern(
            axial_arrangement_patterns=patterns_to_generate,
            available_compositions=comp_pool,
            lp_symmetry_type='rotate_quadrant',
        )
    except (ValueError, RuntimeError) as e:
        print(f"\n[실행 오류] {e}")

NameError: name 'deck' is not defined

In [22]:
deck.card_data

{'JOB_TYP': [{'raw_lines': ['        0       stead                                                           # irrst,jobtyp,\n',
    '        xsl     MAS_XSL\n',
    '        hff     MAS_HFF\n']}],
 'JOB_VER': [{'raw_lines': ['        4.0     2                                                               # xform,ichntyp\n']}],
 'JOB_TIT': [{'raw_lines': ['        Full Core, KNF iSMR Cycle 01, Critical Search\n']}],
 'JOB_IDE': [{'raw_lines': ['        DEPF    1                                                               # iplant,icycle\n']}],
 'JOB_MDL': [{'raw_lines': ['        2       5       0       1       0                                       # irod,iextc,ixsmod,iloc,idch,\n',
    '        0       5                                                               # iben,idete\n']}],
 'GEN_MTH': [{'raw_lines': ['        5       5       0       3       5                                       # imeth,itral,ibndc,ibupco,iasex,\n',
    '        5       5       5       5       5      

In [99]:
class AstraModifier(DeckModifier):
    # 추후 구현 예정

SyntaxError: incomplete input (1552478247.py, line 2)

## 4. DeckWriter

- (250822) 이제 뭘 해야하나,... 수정 완료된 Deck 객체의 카드(.card_data)를 파일에 작성해야함

In [8]:
# deck 객체에 대해서 deck.card_data의 카드타입별 속성이름이 'raw_lines'라면 그대로 작성
# 카드 시퀀스는 등장한 카드이름(그리고 '/' 기호도)을 그대로 붙여놓음
    # 카드가 중복되더라도, 등장한 순서를 카운트한 뒤에 index에 맞는 card_data를 제공하면 됨

class DeckWriter():
    def __init__(self, deck: 'Deck'):
        self.deck = deck
        
    def write_input_file(self, write_file_path='MAS_INP'):
        raise NotImplementedError

In [11]:
deck.card_data

NameError: name 'deck' is not defined

In [25]:
deck.card_sequence

['JOB_TYP',
 'JOB_VER',
 'JOB_TIT',
 'JOB_IDE',
 'JOB_MDL',
 'GEN_MTH',
 'DEF_TFT',
 'GEN_THD',
 'GEN_PIN',
 'EDT_OUT',
 'GEN_DIM',
 'GEN_GEO',
 'GEN_SYM',
 'GEN_CDN',
 'LPD_BCH',
 'LPD_B&C',
 'LPD_C&X',
 'LPD_HFF',
 'ROD_CFG',
 'ROD_MAP',
 'EXE_STD',
 'EXE_DEP',
 'EXE_ROD',
 'EDT_OPT',
 '/',
 'EXE_CRS',
 'EXE_DEP',
 'EDT_OPT',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/',
 'EXE_DEP',
 '/']

In [9]:
class MasterWriter(DeckWriter):
    def __init__(self, deck):
        super().__init__(deck)
        self.writing_method = {
            'GEN_SYM': self._write_gen_sym,
            'GEN_DIM': self._write_gen_dim,
            'LPD_BCH': self._write_lpd_bch,
            'LPD_B&C': self._write_lpd_b_and_c,
            'LPD_C&X': self._write_lpd_c_and_x,
            'LPD_HFF': self._write_lpd_hff,
            'EXE_DEP': self._write_exe_dep
        }

    def write_input_file(self, write_file_path='MAS_INP', core_cycle_option = 'first_cycle'):
        if core_cycle_option == 'first_cycle':
            try:
                with open(write_file_path, 'w', encoding='utf-8') as f:
                    for idx, card_name in enumerate(self.deck.card_sequence):
                        
                        if card_name == '/':
                            f.write('/\n')
                            continue
                        
                        card_idx = sum(1 for x in self.deck.card_sequence[:idx] if x == card_name)
                        card_data = self.deck.card_data[card_name][card_idx]
                        
                        f.write(f'%{card_name}\n')

                        if card_name in self.writing_method:
                            self.writing_method[card_name](f, card_data)
                        elif 'raw_lines' in card_data:
                            for line in card_data['raw_lines']:
                                f.write(line)
                        else:
                            if card_name != '/':
                                print(f"[경고] '{card_name}' 카드의 데이터 형식을 처리할 수 없습니다.", file=sys.stderr)
                    
                    f.write('END')
                    
                print(f"[알림] 파일 '{write_file_path}' 작성 완료")

            except Exception as e:
                print(f"[오류] 파일 작성 중 문제가 발생했습니다: {e}", file=sys.stderr)
                
        else:
            raise NotImplementedError("[오류] 2주기 이후의 경우 파일 쓰기 기능이 아직 구현되지 않았습니다.")

    def _write_gen_sym(self, f, data):
        left = data['left_boundary_condition']
        right = data['right_boundary_condition']
        f.write(f"        {left[0]}       {left[1]}       {left[2]}\n")
        f.write(f"        {right[0]}       {right[1]}       {right[2]}\n")
        
    def _write_gen_dim(self, f, data):

        nx, ny, nz = data['nx'], data['ny'], data['nz']
        nbatch, ncomp = data['nbatch'], data['ncomp']
        
        ndim, ngeo, nsym = data['ndim'], data['ngeo'], data['nsym']
        ndivxy, ndivz = data['ndivxy'], data['ndivz']
        
        ng = data['ng']
        
        f.write(f"        {nx}      {ny}      {nz}      {nbatch}      {ncomp}\n")
        f.write(f"        {ndim}       {ngeo}       {nsym}       {ndivxy}       {ndivz}\n")
        f.write(f"        {ng}\n")
        

    def _write_lpd_bch(self, f, data):
        lp = data['loading_pattern']
        for row in lp:
            # 각 줄의 시작에 8칸 공백을 추가
            line_content = ''.join(f'{cell:>4s}' for cell in row)
            f.write(f"      {line_content}\n")
            
    def _write_lpd_b_and_c(self, f, data):
        sorted_items = sorted(data.items(), key=lambda item: (not item[0].startswith('R'), item[0]))
        
        for batch_id, composition in sorted_items:
            compressed_parts = []
            for key, group in groupby(composition):
                count = len(list(group))
                if count > 1:
                    compressed_parts.append(f"{count}*{key}")
                else:
                    compressed_parts.append(str(key))
            
            composition_str = " ".join(compressed_parts)
            #f.write(f"        {batch_id:<s} {composition_str}\n")
            if batch_id.startswith('R'):
                # R-타입: 11칸 좌측 정렬
                f.write(f"        {batch_id:<11s} {composition_str}\n")
            else:
                # A-타입 및 기타: 8칸 좌측 정렬
                f.write(f"        {batch_id:<8s} {composition_str}\n")

    def _write_lpd_c_and_x(self, f, data):
        sorted_items = sorted(data.items(), key=lambda item: (0, abs(item[0])) if item[0] < 0 else (1, item[0]))

        for idx, name in sorted_items:
            f.write(f"  {idx:>6}      {name:<18s}0\n")

    def _write_lpd_hff(self, f, data):
        for idx, name in sorted(data.items()):
            if idx > 0:
                f.write(f'{idx:>8}       {name}\n')

    def _write_exe_dep(self, f, data):
        delt = data.get('depletion_interval')
        itg = data.get('target_option')
        
        delt_val = int(delt) if delt is not None and delt == int(delt) else delt
        
        f.write(f"{delt_val:>8}       {itg}\n")
        
        if delt is not None and delt < 0:
            sp = data.get('search_parameter')
            sv = data.get('search_value')
            if sp is not None and sv is not None:
                sv_val = int(sv) if sv == int(sv) else sv
                f.write(f"      {sp:<9s}{sv_val}\n")
            
        if itg is not None and itg > 0:
            tk = data.get('target_keff')
            tb = data.get('target_boron')
            if tk is not None and tb is not None:
                 f.write(f"{tk:.5f} {tb:.1f}\n")

In [25]:
writer = MasterWriter(deck)

writer.write_input_file(write_file_path='../MAS_INP_250827')

[알림] 파일 '../MAS_INP_250827' 작성 완료


In [ ]:
class AstraWriter(DeckWriter):
    pass

## 5. SimulationRunner

In [10]:
class SimulationRunner():
    pass

class AstraRunner(SimulationRunner):
    pass

In [11]:
import os
import subprocess
import shutil

class MasterRunner:
    """
    MASTER 프로그램을 실행하고 결과 파일을 관리하는 클래스.
    """
    def run(self, executable_path="master4.0_FNC.exe", input_file="MAS_INP"):
        """
        주어진 입력 파일로 MASTER 실행 파일을 실행합니다.
        """
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"입력 파일 '{input_file}'을 찾을 수 없습니다.")
        
        print(f"[알림] '{executable_path}' 실행 시작...")
        try:
            # os.system보다 subprocess.run이 더 현대적이고 유연합니다.
            # (에러 처리, stdout/stderr 캡처 등이 용이)
            result = subprocess.run(
                [executable_path],
                check=True,  # 실행 실패 시 예외 발생
                capture_output=True,
                text=True
            )
            print(f"[알림] 실행 완료. Return code: {result.returncode}")
            return True
        
        except FileNotFoundError:
            print(f"[오류] 실행 파일 '{executable_path}'을 찾을 수 없습니다.")
            return False
        
        except subprocess.CalledProcessError as e:
            print(f"[오류] 시뮬레이션 실행 중 오류 발생. Return code: {e.returncode}")
            print(f"--- STDOUT ---\n{e.stdout}")
            print(f"--- STDERR ---\n{e.stderr}")
            return False

    def archive_master_results(self, loop_number, base_name="MAS"):
        """
        MASTER 코드 실행 완료 후 결과 파일들의 이름을 변경하여 저장
        """
        print(f"[알림] Loop {loop_number}: 결과 파일 정리 시작...")
        files_to_rename = {
            f"{base_name}_INP": f"{base_name}_INP_loop{loop_number}",
            f"{base_name}_SUM": f"{base_name}_SUM_loop{loop_number}",
            f"{base_name}_OUT": f"{base_name}_OUT_loop{loop_number}",
        }
        
        for old_name, new_name in files_to_rename.items():
            if os.path.exists(old_name):
                try:
                    shutil.move(old_name, new_name) # os.rename 대신 shutil.move 사용 (더 안전)
                    print(f"  - '{old_name}' -> '{new_name}'")
                except Exception as e:
                    print(f"[경고] '{old_name}' 파일 이름 변경 실패: {e}")
            else:
                print(f"[경고] '{old_name}' 파일이 존재하지 않아 이름을 변경할 수 없습니다.")

## 6. DataGeneratrionPipeline

In [12]:
class DataGenerationPipeline:
    AVAILABLE_COMPOSITIONS = [
        '220_000_000', '400_516_000', '400_520_000', '400_720_N04',
        '400_724_704', '495_000_N20', '495_520_508', '495_520_512',
        '495_524_504', '495_720_716', '495_724_712'
    ]
    ALL_AXIAL_PATTERNS = list(MasterModifier.AXIAL_PATTERN_CONFIG.keys())
    FIRST_STAGE_AXIAL_PATTERNS = ['uniform', 'highly_skewed']
    STAGE_STRATEGIES = ['2-stage', 'uniform-random']
    
    def __init__(self, program_name: str, base_input_file: str):
        self.program_name = program_name.upper()
        self.base_input_file = base_input_file
    
        # 4개의 {호출명:함수이름}으로 구성된 딕셔너리를 반환받고, 이를 함수로 사용하기 위해 변수에 할당
        components = self._get_simulation_components(program_name) 
        self.parser = components['parser']
        self.modifier = components['modifier']
        self.writer = components['writer']
        self.runner = components['runner']
    
    @staticmethod
    def _get_simulation_components(program_name):
        if program_name.upper() == 'ASTRA':
            raise ValueError(f"ASTRA의 경우 현재 지원되지 않는 프로그램입니다.")
            # return {
            #     "parser": AstraParser(),
            #     "modifier": AstraModifier(),
            #     "writer": AstraWriter(),
            #     "runner": AstraRunner()
                
            # }
        elif program_name.upper() == 'MASTER':
            return {
                "parser": MasterParser(), # parse_input_file에서 처음으로 Deck 객체가 생성됨
                "modifier": MasterModifier(deck=None), # 따라서 초기화 시점에서는 None으로,
                "writer": MasterWriter(deck=None),
                "runner": MasterRunner()
            }
        else:
            raise ValueError("지원하지 않는 프로그램입니다.")

    def run_pipeline(self,
                     total_samples: int,
                     stage_strategy: str,
                     first_stage_ratio: float = 0.3,
                     octant_symmetry_ratio: float = 0.5,
                     ): # stage ratio의 경우는 스테이지가 존재할 경우에만 받는게 중요할 듯 한데
        
        cases_to_run = []
        base_deck = self.parser.parse_input_file(self.base_input_file) # base deck 객체 생성완료
        
        if stage_strategy == '2-stage':
            # 장전모형 생성시 입력으로 사용될 axial patterns를 생성하는 단계 
            stage1_cases, stage2_cases = self._generate_2_stage_cases(
                total_samples,
                first_stage_ratio,
                octant_symmetry_ratio
            )
            cases_to_run.extend(stage1_cases)
            cases_to_run.extend(stage2_cases)
                
        else:
            axial_patterns = 'uniform-random'
            uniform_random_cases = self._generate_uniform_random_cases(
                total_samples,
                octant_symmetry_ratio
            )
            
            cases_to_run.extend(uniform_random_cases)

        for loop_num in range(total_samples):
            
            current_deck = copy.deepcopy(base_deck)
            
            # modifier
            self.modifier.deck = current_deck
            self.modifier.generate_new_loading_pattern(
                axial_arrangement_patterns=cases_to_run[loop_num]['patterns'],
                available_compositions = self.AVAILABLE_COMPOSITIONS,
                lp_symmetry_type = cases_to_run[loop_num]['symmetry']
            )
            
            # writer
            self.writer.deck = current_deck
            self.writer.write_input_file("MAS_INP")
            self.runner.run(executable_path = 'run_mas.exe')
            self.runner.archive_master_results(loop_num)

                
                
    def _generate_2_stage_cases(self,
                                  total_samples,
                                  first_stage_ratio,
                                  octant_symmetry_ratio
                                  ) -> tuple[list, list]:
        """
        [2-stage 케이스 생성용]
        총 샘플 수 및 stage 비율을 입력받은 후, 사전에 axial_patterns의 케이스들에 대한 리스트를 작성
        """
        stage1_cases, stage2_cases = [], []
        stage1_patterns = self.FIRST_STAGE_AXIAL_PATTERNS # ['uniform', 'highly_skewed']
        num_stage1_cases = int(total_samples*first_stage_ratio)
        num_stage2_cases = total_samples - num_stage1_cases

        for _ in range(num_stage1_cases):
        
            num_fuel_batches = random.randint(2, 3) # stage1에는 2~3종의 연료 집합체 허용
            # 정해진 num_fuel_batches 개수만큼 중복을 허용하여 stage1_patterns에서 무작위로 뽑음
                # stage1_patterns: 가능한 패턴은 uniform 또는 highly_skewed
            pattern_case = random.choices(stage1_patterns, k = num_fuel_batches)
            # NOTE: MasterModifier(DeckModifier) 내부에 명시된 가능한 옵션은 'rotate_octant' 또는 'rotate_quadrant' 라고 지정됨
            # 하지만 'octant_symmetry' 또는 'quadrant_symmetry'도 가능하도록 항목을 변환할 수 있도록 기능 추가 (SYMMETRY_ALIASES 참고)
            symmetry = "octant_symmetry" if random.random() < octant_symmetry_ratio else "quadrant_symmetry"
        
            case = {
                'strategy': '2-stage',
                'stage': 1, 
                'patterns': pattern_case,
                'symmetry': symmetry
            }
            
            stage1_cases.append(case)
            
        for _ in range(num_stage2_cases):
        
            # 가능한 패턴은 2 ~ 5
            num_fuel_batches = random.randint(2, 5)
        
            # 만약 num_fuel_batches가 2 또는 3이라면
            if num_fuel_batches in (2,3):
                stage2_patterns = list(set(self.ALL_AXIAL_PATTERNS) - set(self.FIRST_STAGE_AXIAL_PATTERNS))
                pattern_case = random.choices(stage2_patterns, k = num_fuel_batches)
            
            else: # 4 or 5라면
                stage2_patterns = self.ALL_AXIAL_PATTERNS
                pattern_case = random.choices(stage2_patterns, k = num_fuel_batches)

            symmetry = "octant_symmetry" if random.random() < octant_symmetry_ratio else "quadrant_symmetry"
            
            case = {
                'strategy': '2-stage',
                'stage': 2,
                'patterns': pattern_case,
                'symmetry': symmetry
            }
            
            stage2_cases.append(case)
            
        return stage1_cases, stage2_cases
    

    def _generate_uniform_random_cases(self,
                                        total_samples,
                                        octant_symmetry_ratio,
                                        ) -> list:
        
        uniform_random_cases = []
        
        for _ in range(total_samples):
            num_fuel_batches = random.randint(2, 5)
            pattern_case = random.choices(self.ALL_AXIAL_PATTERNS, k= num_fuel_batches)
            symmetry = "octant_symmetry" if random.random() < octant_symmetry_ratio else "quadrant_symmetry"
            
            case = {
                'strategy': 'uniform-random',
                'stage': None,
                'patterns': pattern_case,
                'symmetry': symmetry
            }
            
            uniform_random_cases.append(case)
            
        return uniform_random_cases
            
    

### 문제있음!!! (+병렬처리 기능 9월 1일 월요일에 넣을 것)
- Loop대로 데이터 생성하는건 가능하나, 루프 1 -> 2로 넘어가기 전에
- 이전 루프의 MAS_RST나 MAS_PPI가 작성 완료되지 않고 루프가 넘어가면서 오류 발생
- 지금은, EDT_OPT 부분에서 파일 작성 옵션을 끄고 실행하면 되는데,
- 나중에 탄력운전이나 재시작(RST) 기능 활용할 때는 파일을 동적으로 인식해서 완료되기 전까지 기다리는 방식 또는 메서드 작성 필요  

In [16]:
pipeline = DataGenerationPipeline(program_name="MASTER", base_input_file='DEP_iSMR_DPL1_F01.INP')

In [17]:
pipeline.run_pipeline(total_samples=10, stage_strategy='2-stage')

[알림] 새로운 장전 패턴 생성: 핵연료 집합체 2종, 대칭 조건='quadrant_symmetry', 축방향 배열 방식: ['uniform', 'highly_skewed']
  └─ 단계 1: 핵종 조성 할당 시작...
    ├─ 필요한 역할: 특별 주 조성=2, 일반 주 조성=0, 보조 조성=1
    └─ [시도 01] 성공! 모든 제약 조건을 만족하는 할당을 찾았습니다.
        - A1 (uniform): ['495_520_508']
        - A2 (highly_skewed): ['495_000_N20', '220_000_000']
  ├─ 연료 조성 인덱스: 3 종류
  ├─ 기존 반사체 조성 정보 보존: 7 종류
  └─ 최종 조성 맵 완성: 총 10 종류 (반사체 + 연료)
    ├─ 카드 데이터 상세:
    │ [반사체] -1: REF_AXIAL
    │ [반사체] -2: REF_R1
    │ [반사체] -3: REF_R2
    │ [반사체] -4: REF_R3
    │ [반사체] -5: REF_R4
    │ [반사체] -6: REF_R5
    │ [반사체] -7: REF_R6
    │ [연료]   1: 220_000_000
    │ [연료]   2: 495_000_N20
    │ [연료]   3: 495_520_508
    └─ 카드 데이터 출력 완료.
  ├─ 'LPD_C&X' 카드의 데이터를 [새로운 데이터로 교체]합니다.
  └─ 'LPD_C&X' 카드 업데이트 완료.
  ├─ 'LPD_HFF' 카드의 데이터를 [반사체를 제외한 연료 조성(composition)만 교체]합니다.
  └─ 'LPD_HFF' 카드 업데이트 완료.
  ├─ 'LPD_B&C' 카드의 데이터를 [새로운 데이터로 교체]합니다.
  └─ 'LPD_B&C' 카드 업데이트 완료.
    ├─ 노심 크기 정의: FULL_CORE=11x11, QUATER_CORE=6x6
  ├─ 'LPD_BCH' 카드의 데이터를 [새로운 데이터로 교체]

In [105]:
pipeline = DataGenerationPipeline()

TypeError: DataGenerationPipeline.__init__() missing 2 required positional arguments: 'program_name' and 'base_input_file'